In [ ]:
# Abfrage von OSM-Daten aus overpass-API, Parallele Segmente in DB Rad+ Daten finden, optional neues Attribut in Rad+ Datensatz aufnehmen
# Zur Erstellung des Codes wurde die generative Künstliche Intelligenz (KI) „Claude AI“ des Anbieters Anthropic in Version 3.7 genutzt

In [2]:
# Step 1: var 1 Request data from overpass API (one Street Name, TEST Kaiserdamm)
# Adjust Overpass API query and test in Overpass Turbo before launching
import requests
import json
import pandas as pd
import geopandas as gpd
import pyogrio
from shapely.geometry import Point, LineString
import urllib.parse

def get_osm_data(query):
    """
    Send query to Overpass API and return JSON response
    """
    encoded_query = urllib.parse.quote(query)
    overpass_url = "http://overpass-api.de/api/interpreter"
    
    response = requests.get(overpass_url, params={'data': query})
    
    if response.status_code != 200:
        raise Exception(f"Request failed with status code: {response.status_code}")
        
    return response.json()

def process_osm_to_gdf(data):
    """
    Convert OSM JSON data to GeoDataFrame using efficient methods
    """
    # Extract nodes and ways
    nodes = {
        element['id']: Point(element['lon'], element['lat'])
        for element in data['elements']
        if element['type'] == 'node'
    }
    
    ways = [
        element for element in data['elements']
        if element['type'] == 'way'
    ]
    
    # Create features list with proper structure
    features = []
    
    # Process each way
    for way in ways:
        coords = []
        for node_id in way['nodes']:
            if node_id in nodes:
                point = nodes[node_id]
                coords.append((point.x, point.y))
        
        if coords:
            line = LineString(coords)
            # Create a dictionary with all properties and geometry
            feature = way.get('tags', {}).copy()  # Start with tags
            feature['id'] = way['id']
            feature['type'] = 'way'
            feature['geometry'] = line
            features.append(feature)
    
    # Create DataFrame first
    if not features:
        raise ValueError("No features found in the OSM data")
        
    # Create GeoDataFrame with explicit geometry column
    gdf = gpd.GeoDataFrame(
        features,
        geometry='geometry',
        crs='EPSG:4326'
    )
    
    # Transform to EPSG:25833
    gdf = gdf.to_crs(epsg=25833)
    
    return gdf

# Your query
query = """
[out:json][timeout:25];
// Get Berlin using its relation ID
area(3600062422)->.berlin;  // 62422 is Berlin's relation ID
// Get the street only within Berlin area
way["name"="Kaiserdamm"](area.berlin)->.mainStreet;
(
  // The main street
  way.mainStreet;
  // All nodes of the main street
  node(w.mainStreet);
);
out body;
out geom;
"""

try:
    osm_data = get_osm_data(query)
    gdf = process_osm_to_gdf(osm_data)
    
    print("\nGeoDataFrame Head:")
    print(gdf.head())
    
    print("\nColumns in GeoDataFrame:")
    print(gdf.columns.tolist())
    
    print("\nCRS Information:")
    print(gdf.crs)
    
    # Optional: Save to file
    # pyogrio.write_dataframe(
    #     gdf,
    #     "kaiserdamm.gpkg",
    #     driver="GPKG",
    #     layer="street"
    # )
    
except Exception as e:
    print(f"Error: {str(e)}")


GeoDataFrame Head:
             bicycle cycleway:left cycleway:right          foot  highway  \
0  optional_sidepath            no       separate  use_sidepath  primary   
1  optional_sidepath            no       separate  use_sidepath  primary   
2  optional_sidepath            no       separate  use_sidepath  primary   
3  optional_sidepath            no       separate  use_sidepath  primary   
4  optional_sidepath            no       separate  use_sidepath  primary   

  lanes  lit maxspeed        name name:etymology:wikidata  ... destination  \
0     4  yes       50  Kaiserdamm                   Q2677  ...         NaN   
1     4  yes       50  Kaiserdamm                   Q2677  ...         NaN   
2     4  yes       50  Kaiserdamm                   Q2677  ...         NaN   
3     4  yes       50  Kaiserdamm                   Q2677  ...         NaN   
4     4  yes       50  Kaiserdamm                   Q2677  ...         NaN   

  destination:colour destination:ref:to destination:sy

In [1]:
# Step 1: var 2 Request data from overpass API (Multiple Street names, Wallstraßen)
# Adjust Overpass API query and test in Overpass Turbo before launching
import requests
import json
import pandas as pd
import geopandas as gpd
import pyogrio
from shapely.geometry import Point, LineString
import urllib.parse

def get_osm_data(query):
    """
    Send query to Overpass API and return JSON response
    """
    encoded_query = urllib.parse.quote(query)
    overpass_url = "http://overpass-api.de/api/interpreter"
    
    response = requests.get(overpass_url, params={'data': query})
    
    if response.status_code != 200:
        raise Exception(f"Request failed with status code: {response.status_code}")
        
    return response.json()

def process_osm_to_gdf(data):
    """
    Convert OSM JSON data to GeoDataFrame using efficient methods
    """
    # Extract nodes and ways
    nodes = {
        element['id']: Point(element['lon'], element['lat'])
        for element in data['elements']
        if element['type'] == 'node'
    }
    
    ways = [
        element for element in data['elements']
        if element['type'] == 'way'
    ]
    
    # Create features list with proper structure
    features = []
    
    # Process each way
    for way in ways:
        coords = []
        for node_id in way['nodes']:
            if node_id in nodes:
                point = nodes[node_id]
                coords.append((point.x, point.y))
        
        if coords:
            line = LineString(coords)
            # Create a dictionary with all properties and geometry
            feature = way.get('tags', {}).copy()  # Start with tags
            feature['id'] = way['id']
            feature['type'] = 'way'
            feature['geometry'] = line
            features.append(feature)
    
    # Create DataFrame first
    if not features:
        raise ValueError("No features found in the OSM data")
        
    # Create GeoDataFrame with explicit geometry column
    gdf = gpd.GeoDataFrame(
        features,
        geometry='geometry',
        crs='EPSG:4326'
    )
    
    # Transform to EPSG:25833
    gdf = gdf.to_crs(epsg=25833)
    
    return gdf

# Your query
query = """
[out:json][timeout:25];
// Get Berlin using its relation ID
area(3600062422)->.berlin;  // 62422 is Berlin's relation ID
// Get multiple streets within Berlin area and their nodes
(
  // Remove ["bicycle_road"="yes"] at Wallstraße to include street infront of Märkisches Museum (no bicycle road)
  way["name"="Wallstraße"]["bicycle_road"="yes"](area.berlin);
  way["name"="Oberwallstraße"](area.berlin);
  way["name"="Niederwallstraße"](area.berlin);
  way["name"="Märkisches Ufer"]["bicycle_road"="yes"](area.berlin);
);
out body;
>;
out geom;
"""

try:
    osm_data = get_osm_data(query)
    gdf = process_osm_to_gdf(osm_data)
    
    print("\nGeoDataFrame Head:")
    print(gdf.head())
    
    print("\nColumns in GeoDataFrame:")
    print(gdf.columns.tolist())
    
    print("\nCRS Information:")
    print(gdf.crs)
    
    # Optional: Save to file
    # pyogrio.write_dataframe(
    #     gdf,
    #     "kaiserdamm.gpkg",
    #     driver="GPKG",
    #     layer="street"
    # )
    
except Exception as e:
    print(f"Error: {str(e)}")


GeoDataFrame Head:
  cycleway:both      highway  \
0            no  residential   
1            no  residential   
2            no  residential   
3            no  residential   
4            no  residential   

                                               image lane_markings  lit  \
0  https://commons.wikimedia.org/wiki/File:Mitte_...            no  yes   
1  https://commons.wikimedia.org/wiki/File:Mitte_...            no  yes   
2  https://commons.wikimedia.org/wiki/File:Mitte_...           NaN  yes   
3                                                NaN           NaN  yes   
4                                                NaN            no  yes   

  maxspeed maxspeed:type              name name:etymology:wikidata  \
0       30     DE:zone30    Oberwallstraße                  Q91203   
1       30           NaN    Oberwallstraße                  Q91203   
2       30           NaN  Niederwallstraße                Q1398534   
3       30     DE:zone30        Wallstraße              

In [3]:
# Step 1: var 3 Request data from overpass API (Multiple Street names, Handjerystraße)
# Adjust Overpass API query and test in Overpass Turbo before launching
import requests
import json
import pandas as pd
import geopandas as gpd
import pyogrio
from shapely.geometry import Point, LineString
import urllib.parse

def get_osm_data(query):
    """
    Send query to Overpass API and return JSON response
    """
    encoded_query = urllib.parse.quote(query)
    overpass_url = "http://overpass-api.de/api/interpreter"
    
    response = requests.get(overpass_url, params={'data': query})
    
    if response.status_code != 200:
        raise Exception(f"Request failed with status code: {response.status_code}")
        
    return response.json()

def process_osm_to_gdf(data):
    """
    Convert OSM JSON data to GeoDataFrame using efficient methods
    """
    # Extract nodes and ways
    nodes = {
        element['id']: Point(element['lon'], element['lat'])
        for element in data['elements']
        if element['type'] == 'node'
    }
    
    ways = [
        element for element in data['elements']
        if element['type'] == 'way'
    ]
    
    # Create features list with proper structure
    features = []
    
    # Process each way
    for way in ways:
        coords = []
        for node_id in way['nodes']:
            if node_id in nodes:
                point = nodes[node_id]
                coords.append((point.x, point.y))
        
        if coords:
            line = LineString(coords)
            # Create a dictionary with all properties and geometry
            feature = way.get('tags', {}).copy()  # Start with tags
            feature['id'] = way['id']
            feature['type'] = 'way'
            feature['geometry'] = line
            features.append(feature)
    
    # Create DataFrame first
    if not features:
        raise ValueError("No features found in the OSM data")
        
    # Create GeoDataFrame with explicit geometry column
    gdf = gpd.GeoDataFrame(
        features,
        geometry='geometry',
        crs='EPSG:4326'
    )
    
    # Transform to EPSG:25833
    gdf = gdf.to_crs(epsg=25833)
    
    return gdf

# Your query
query = """
[out:json][timeout:25];
// Define the Berlin area
area(3600062422)->.berlin;

// Find the Tempelhof-Schöneberg administrative area
rel["admin_level"="9"]["name"="Tempelhof-Schöneberg"](area.berlin)->.tempelhof;

// Convert the relation to an area
.tempelhof map_to_area -> .tempelhof_area;

// Get the streets within Tempelhof-Schöneberg
// Remove ["bicycle_road"="yes"] to include Kreuzung Schmiljanstraße (No Cycling Street)
(
  way(area.tempelhof_area)["name"="Handjerystraße"]["bicycle_road"="yes"];
  // Optional: Renée-Sintenis-Platz
  way(area.tempelhof_area)["name:left"="Renée-Sintenis-Platz"]["bicycle_road"="yes"];
  way(area.tempelhof_area)["name:right"="Handjerystraße"]["bicycle_road"="yes"];
);
out body;
>;
out geom;
"""

try:
    osm_data = get_osm_data(query)
    gdf = process_osm_to_gdf(osm_data)
    
    print("\nGeoDataFrame Head:")
    print(gdf.head())
    
    print("\nColumns in GeoDataFrame:")
    print(gdf.columns.tolist())
    
    print("\nCRS Information:")
    print(gdf.crs)
    
    # Optional: Save to file
    # pyogrio.write_dataframe(
    #     gdf,
    #     "kaiserdamm.gpkg",
    #     driver="GPKG",
    #     layer="street"
    # )
    
except Exception as e:
    print(f"Error: {str(e)}")


GeoDataFrame Head:
      bicycle bicycle_road          foot      highway lane_markings  lit  \
0  designated          yes  use_sidepath  residential            no  yes   
1  designated          yes  use_sidepath  residential            no  yes   
2  designated          yes  use_sidepath  residential            no  yes   
3  designated          yes  use_sidepath  residential           NaN  yes   
4  designated          yes  use_sidepath  residential            no  yes   

  maxspeed                                   name name:etymology:wikidata  \
0       30                         Handjerystraße                Q1986912   
1       30                         Handjerystraße                Q1986912   
2       30                         Handjerystraße                Q1986912   
3       30  Handjerystraße / Renée-Sintenis-Platz                     NaN   
4       30                         Handjerystraße                Q1986912   

  parking:left  ... motor_vehicle             name:left  \
0

In [5]:
# Step 1: var 4 Request data from overpass API (Multiple Street names, Stallschreiberstraße)
# Adjust Overpass API query and test in Overpass Turbo before launching
import requests
import json
import pandas as pd
import geopandas as gpd
import pyogrio
from shapely.geometry import Point, LineString
import urllib.parse

def get_osm_data(query):
    """
    Send query to Overpass API and return JSON response
    """
    encoded_query = urllib.parse.quote(query)
    overpass_url = "http://overpass-api.de/api/interpreter"
    
    response = requests.get(overpass_url, params={'data': query})
    
    if response.status_code != 200:
        raise Exception(f"Request failed with status code: {response.status_code}")
        
    return response.json()

def process_osm_to_gdf(data):
    """
    Convert OSM JSON data to GeoDataFrame using efficient methods
    """
    # Extract nodes and ways
    nodes = {
        element['id']: Point(element['lon'], element['lat'])
        for element in data['elements']
        if element['type'] == 'node'
    }
    
    ways = [
        element for element in data['elements']
        if element['type'] == 'way'
    ]
    
    # Create features list with proper structure
    features = []
    
    # Process each way
    for way in ways:
        coords = []
        for node_id in way['nodes']:
            if node_id in nodes:
                point = nodes[node_id]
                coords.append((point.x, point.y))
        
        if coords:
            line = LineString(coords)
            # Create a dictionary with all properties and geometry
            feature = way.get('tags', {}).copy()  # Start with tags
            feature['id'] = way['id']
            feature['type'] = 'way'
            feature['geometry'] = line
            features.append(feature)
    
    # Create DataFrame first
    if not features:
        raise ValueError("No features found in the OSM data")
        
    # Create GeoDataFrame with explicit geometry column
    gdf = gpd.GeoDataFrame(
        features,
        geometry='geometry',
        crs='EPSG:4326'
    )
    
    # Transform to EPSG:25833
    gdf = gdf.to_crs(epsg=25833)
    
    return gdf

# Your query
query = """
[out:json][timeout:25];
// Get Berlin using its relation ID
area(3600062422)->.berlin;  // 62422 is Berlin's relation ID
// Get multiple streets within Berlin area and their nodes
(
  way["name"="Stallschreiberstraße"]["bicycle_road"="yes"](area.berlin);
);
out body;
>;
out geom;
"""

try:
    osm_data = get_osm_data(query)
    gdf = process_osm_to_gdf(osm_data)
    
    print("\nGeoDataFrame Head:")
    print(gdf.head())
    
    print("\nColumns in GeoDataFrame:")
    print(gdf.columns.tolist())
    
    print("\nCRS Information:")
    print(gdf.crs)
    
    # Optional: Save to file
    # pyogrio.write_dataframe(
    #     gdf,
    #     "kaiserdamm.gpkg",
    #     driver="GPKG",
    #     layer="street"
    # )
    
except Exception as e:
    print(f"Error: {str(e)}")


GeoDataFrame Head:
      bicycle bicycle_road cycleway:both          foot      highway  \
0  designated          yes            no  use_sidepath  residential   
1  designated          yes            no  use_sidepath  residential   
2  designated          yes            no  use_sidepath  residential   
3  designated          yes            no  use_sidepath  residential   
4  designated          yes            no  use_sidepath  residential   

                                               image lane_markings  lit  \
0  https://commons.wikimedia.org/wiki/File:Berlin...            no  yes   
1  https://commons.wikimedia.org/wiki/File:Berlin...            no  yes   
2  https://commons.wikimedia.org/wiki/File:Berlin...            no  yes   
3  https://commons.wikimedia.org/wiki/File:Berlin...            no  yes   
4  https://commons.wikimedia.org/wiki/File:Berlin...            no  yes   

  maxspeed                  name  ... parking:both:reason sidewalk:both  \
0       30  Stallschreibers

In [7]:
# Step 1: var 5 Request data from overpass API (Multiple Street names, Hönower Weg)
# Adjust Overpass API query and test in Overpass Turbo before launching
import requests
import json
import pandas as pd
import geopandas as gpd
import pyogrio
from shapely.geometry import Point, LineString
import urllib.parse

def get_osm_data(query):
    """
    Send query to Overpass API and return JSON response
    """
    encoded_query = urllib.parse.quote(query)
    overpass_url = "http://overpass-api.de/api/interpreter"
    
    response = requests.get(overpass_url, params={'data': query})
    
    if response.status_code != 200:
        raise Exception(f"Request failed with status code: {response.status_code}")
        
    return response.json()

def process_osm_to_gdf(data):
    """
    Convert OSM JSON data to GeoDataFrame using efficient methods
    """
    # Extract nodes and ways
    nodes = {
        element['id']: Point(element['lon'], element['lat'])
        for element in data['elements']
        if element['type'] == 'node'
    }
    
    ways = [
        element for element in data['elements']
        if element['type'] == 'way'
    ]
    
    # Create features list with proper structure
    features = []
    
    # Process each way
    for way in ways:
        coords = []
        for node_id in way['nodes']:
            if node_id in nodes:
                point = nodes[node_id]
                coords.append((point.x, point.y))
        
        if coords:
            line = LineString(coords)
            # Create a dictionary with all properties and geometry
            feature = way.get('tags', {}).copy()  # Start with tags
            feature['id'] = way['id']
            feature['type'] = 'way'
            feature['geometry'] = line
            features.append(feature)
    
    # Create DataFrame first
    if not features:
        raise ValueError("No features found in the OSM data")
        
    # Create GeoDataFrame with explicit geometry column
    gdf = gpd.GeoDataFrame(
        features,
        geometry='geometry',
        crs='EPSG:4326'
    )
    
    # Transform to EPSG:25833
    gdf = gdf.to_crs(epsg=25833)
    
    return gdf

# Your query
query = """
[out:json][timeout:25];
// Get Berlin using its relation ID
area(3600062422)->.berlin;  // 62422 is Berlin's relation ID
// Get multiple streets within Berlin area and their nodes
(
  // Remove ["bicycle_road"="yes"] to include Road infront of Parkplatz Dolgenseestraße (No bicycle road)
  way["name"="Hönower Weg"]["bicycle_road"="yes"](area.berlin);
);
out body;
>;
out geom;
"""

try:
    osm_data = get_osm_data(query)
    gdf = process_osm_to_gdf(osm_data)
    
    print("\nGeoDataFrame Head:")
    print(gdf.head())
    
    print("\nColumns in GeoDataFrame:")
    print(gdf.columns.tolist())
    
    print("\nCRS Information:")
    print(gdf.crs)
    
    # Optional: Save to file
    # pyogrio.write_dataframe(
    #     gdf,
    #     "kaiserdamm.gpkg",
    #     driver="GPKG",
    #     layer="street"
    # )
    
except Exception as e:
    print(f"Error: {str(e)}")


GeoDataFrame Head:
      bicycle bicycle_road cycleway:both      highway lane_markings  lit  \
0  designated          yes            no  residential            no  yes   
1  designated          yes           NaN      service           NaN  yes   
2  designated          yes            no  residential            no  yes   
3  designated          yes            no  residential            no   no   
4  designated          yes            no  residential            no   no   

  maxspeed         name name:etymology:wikidata name:etymology:wikipedia  ...  \
0       30  Hönower Weg                 Q176778                 de:Hönow  ...   
1       30  Hönower Weg                     NaN                      NaN  ...   
2       30  Hönower Weg                 Q176778                 de:Hönow  ...   
3       30  Hönower Weg                 Q176778                 de:Hönow  ...   
4       30  Hönower Weg                 Q176778                 de:Hönow  ...   

                                    

In [1]:
# Step 1.1: var 6 Request data from overpass API (custom, geschützte Radfahrstreifen)
# Adjust Overpass API query and test in Overpass Turbo before launching
import requests
import json
import pandas as pd
import geopandas as gpd
import pyogrio
from shapely.geometry import Point, LineString

def get_osm_data(query):
    """
    Send query to Overpass API and return JSON response
    """
    overpass_url = "https://overpass-api.de/api/interpreter"
    
    # Use POST method with the query directly in the data parameter
    response = requests.post(overpass_url, data=query)
    
    if response.status_code != 200:
        raise Exception(f"Request failed with status code: {response.status_code}, Response: {response.text[:200]}")
        
    return response.json()

def process_osm_to_gdf(data):
    """
    Convert OSM JSON data to GeoDataFrame using efficient methods
    """
    # Extract nodes and ways
    nodes = {
        element['id']: Point(element['lon'], element['lat'])
        for element in data['elements']
        if element['type'] == 'node'
    }
    
    ways = [
        element for element in data['elements']
        if element['type'] == 'way'
    ]
    
    # Create features list with proper structure
    features = []
    
    # Process each way
    for way in ways:
        coords = []
        # Handle ways with embedded geometry
        if 'geometry' in way:
            for node in way['geometry']:
                coords.append((node['lon'], node['lat']))
        # Handle ways with node references
        elif 'nodes' in way:
            for node_id in way['nodes']:
                if node_id in nodes:
                    point = nodes[node_id]
                    coords.append((point.x, point.y))
        
        if coords:
            line = LineString(coords)
            # Create a dictionary with all properties and geometry
            feature = way.get('tags', {}).copy()  # Start with tags
            feature['id'] = way['id']
            feature['type'] = 'way'
            feature['geometry'] = line
            features.append(feature)
    
    # Create DataFrame first
    if not features:
        raise ValueError("No features found in the OSM data")
        
    # Create GeoDataFrame with explicit geometry column
    gdf = gpd.GeoDataFrame(
        features,
        geometry='geometry',
        crs='EPSG:4326'
    )
    
    # Transform to EPSG:25833
    gdf = gdf.to_crs(epsg=25833)
    
    return gdf

# Modified query using Berlin's relation ID directly
query = """
[out:json][timeout:100];
// Get Berlin area by its relation ID (62422 is Berlin's OSM relation ID)
area(3600062422)->.searchArea;
(
  // Roads with cycleway:right:separation:left = bump or bollard
  way["highway"]
      ["cycleway:right:separation:left"~"bump|bollard"]
      (area.searchArea);
  // Cycleways with separation:left = bollard
  way["highway"="cycleway"]
      ["separation:left"~"bump|bollard"]
      (area.searchArea);
);
// Output results as individual road segments
out geom;
"""

# Alternative query if you prefer to use name-based area search
alternative_query = """
[out:json][timeout:100];
// Define Berlin's boundary
rel["name"="Berlin"]["admin_level"="4"]["boundary"="administrative"];
map_to_area->.searchArea;
(
  // Roads with cycleway:right:separation:left = bump or bollard
  way["highway"]
      ["cycleway:right:separation:left"~"bump|bollard"]
      (area.searchArea);
  // Cycleways with separation:left = bollard
  way["highway"="cycleway"]
      ["separation:left"~"bump|bollard"]
      (area.searchArea);
);
// Output results as individual road segments
out geom;
"""

try:
    # Use the first query with relation ID (more reliable)
    osm_data = get_osm_data(query)
    gdf = process_osm_to_gdf(osm_data)
    
    print(f"\nFound {len(gdf)} features")
    
    print("\nGeoDataFrame Head:")
    print(gdf.head())
    
    print("\nColumns in GeoDataFrame:")
    print(gdf.columns.tolist())
    
    print("\nCRS Information:")
    print(gdf.crs)
    
    # Optional: Save to file
    # pyogrio.write_dataframe(
    #     gdf,
    #     "protected_bike_lanes.gpkg",
    #     driver="GPKG",
    #     layer="protected_bike_lanes"
    # )
    
except Exception as e:
    print(f"Error: {str(e)}")
    
    # If the first query fails, try the alternative
    print("\nFirst query failed. Trying alternative query...")
    try:
        osm_data = get_osm_data(alternative_query)
        gdf = process_osm_to_gdf(osm_data)
        
        print(f"\nFound {len(gdf)} features with alternative query")
        
        print("\nGeoDataFrame Head:")
        print(gdf.head())
        
    except Exception as e2:
        print(f"Alternative query also failed: {str(e2)}")


Found 158 features

GeoDataFrame Head:
  cycleway:left cycleway:right cycleway:right:bicycle cycleway:right:lane  \
0            no           lane             designated           exclusive   
1            no           lane                    NaN           exclusive   
2            no           lane             designated           exclusive   
3            no           lane             designated           exclusive   
4            no           lane                    NaN           exclusive   

  cycleway:right:separation:left cycleway:right:width          foot  \
0                           bump                  2.8  use_sidepath   
1                           bump                  NaN  use_sidepath   
2           bollard;parking_lane                    2  use_sidepath   
3                        bollard                    2  use_sidepath   
4            bump;vertical_panel                  2.4  use_sidepath   

     highway lanes  lit  ... buffer:both lanes:bus class:bicycle  \
0 

In [ ]:
# Step 1.1: var 7 Request data from overpass API (Fahrradstraßen)
# Adjust Overpass API query and test in Overpass Turbo before launching
import requests
import json
import pandas as pd
import geopandas as gpd
import pyogrio
from shapely.geometry import Point, LineString

def get_osm_data(query):
    """
    Send query to Overpass API and return JSON response
    """
    overpass_url = "https://overpass-api.de/api/interpreter"
    
    # Use POST method with the query directly in the data parameter
    response = requests.post(overpass_url, data=query)
    
    if response.status_code != 200:
        raise Exception(f"Request failed with status code: {response.status_code}, Response: {response.text[:200]}")
        
    return response.json()

def process_osm_to_gdf(data):
    """
    Convert OSM JSON data to GeoDataFrame using efficient methods
    """
    # Extract nodes and ways
    nodes = {
        element['id']: Point(element['lon'], element['lat'])
        for element in data['elements']
        if element['type'] == 'node'
    }
    
    ways = [
        element for element in data['elements']
        if element['type'] == 'way'
    ]
    
    # Create features list with proper structure
    features = []
    
    # Process each way
    for way in ways:
        coords = []
        # Handle ways with embedded geometry
        if 'geometry' in way:
            for node in way['geometry']:
                coords.append((node['lon'], node['lat']))
        # Handle ways with node references
        elif 'nodes' in way:
            for node_id in way['nodes']:
                if node_id in nodes:
                    point = nodes[node_id]
                    coords.append((point.x, point.y))
        
        if coords:
            line = LineString(coords)
            # Create a dictionary with all properties and geometry
            feature = way.get('tags', {}).copy()  # Start with tags
            feature['id'] = way['id']
            feature['type'] = 'way'
            feature['geometry'] = line
            features.append(feature)
    
    # Create DataFrame first
    if not features:
        raise ValueError("No features found in the OSM data")
        
    # Create GeoDataFrame with explicit geometry column
    gdf = gpd.GeoDataFrame(
        features,
        geometry='geometry',
        crs='EPSG:4326'
    )
    
    # Transform to EPSG:25833
    gdf = gdf.to_crs(epsg=25833)
    
    return gdf

# Modified query using Berlin's relation ID directly
query = """
[out:json][timeout:25];
area[name="Berlin"]->.searchArea;
(
  
  // Straßen, die als Wohnstraße markiert sind und für Fahrräder ausgewiesen
  way["highway"="residential"]["bicycle"~"designated|official"](area.searchArea);
  
  // Straßen, die möglicherweise eine Verkehrsberuhigung für Fahrräder haben
  way["bicycle_road"="yes"](area.searchArea);
);
out body;
>;
out skel qt;
"""

# Alternative query if you prefer to use name-based area search
alternative_query = """
[out:json][timeout:100];
// Define Berlin's boundary
rel["name"="Berlin"]["admin_level"="4"]["boundary"="administrative"];
map_to_area->.searchArea;
(
  // Roads with cycleway:right:separation:left = bump or bollard
  way["highway"]
      ["cycleway:right:separation:left"~"bump|bollard"]
      (area.searchArea);
  // Cycleways with separation:left = bollard
  way["highway"="cycleway"]
      ["separation:left"~"bump|bollard"]
      (area.searchArea);
);
// Output results as individual road segments
out geom;
"""

try:
    # Use the first query with relation ID (more reliable)
    osm_data = get_osm_data(query)
    gdf = process_osm_to_gdf(osm_data)
    
    print(f"\nFound {len(gdf)} features")
    
    print("\nGeoDataFrame Head:")
    print(gdf.head())
    
    print("\nColumns in GeoDataFrame:")
    print(gdf.columns.tolist())
    
    print("\nCRS Information:")
    print(gdf.crs)
    
    # Optional: Save to file
    # pyogrio.write_dataframe(
    #     gdf,
    #     "protected_bike_lanes.gpkg",
    #     driver="GPKG",
    #     layer="protected_bike_lanes"
    # )
    
except Exception as e:
    print(f"Error: {str(e)}")
    
    # If the first query fails, try the alternative
    print("\nFirst query failed. Trying alternative query...")
    try:
        osm_data = get_osm_data(alternative_query)
        gdf = process_osm_to_gdf(osm_data)
        
        print(f"\nFound {len(gdf)} features with alternative query")
        
        print("\nGeoDataFrame Head:")
        print(gdf.head())
        
    except Exception as e2:
        print(f"Alternative query also failed: {str(e2)}")

In [9]:
# 1.1: Check if necessary
# gdf.explore()

# Create a simplified GeoDataFrame with only id and geometry
gdf_simple = gdf[['id', 'geometry']].copy()

# Create the interactive map
gdf_simple.explore()

In [2]:
# Step 2 v2.3 final [FULL] [parquet]: Find parallel segments to the overpass data in analysis data
# ADJUST buffer_distance and parallel_threshold at the end of the code!

import geopandas as gpd
import pandas as pd
import pyogrio
import numpy as np
from shapely.geometry import LineString, Point
from shapely import wkt
import pyarrow.parquet as pq
from sklearn.linear_model import LinearRegression  # For fitting a line to vertices
from shapely.geometry import CAP_STYLE

def find_parallel_segments(gdf, parquet_path, buffer_distance=15, parallel_threshold=0.8):
    """
    Find parallel segments from a parquet file with WKT geometries within a buffer of the input geometry.
    
    Parameters:
    -----------
    gdf : GeoDataFrame
        Input GeoDataFrame containing the reference street geometries (selection tool)
    parquet_path : str
        Path to the parquet file with WKT geometries (data source)
    buffer_distance : float
        Buffer distance in meters (default: 15)
    parallel_threshold : float
        Threshold for parallel score (default: 0.8)
        
    Returns:
    --------
    tuple : (nodes_in_buffer, parallel_nodes)
        Two GeoDataFrames containing all nodes in buffer and parallel nodes
    """
    
    # Create buffer for each reference geometry
    gdf_buffer = gdf.copy()
    # gdf_buffer['geometry'] = gdf.geometry.buffer(buffer_distance)
    gdf_buffer['geometry'] = gdf.geometry.apply(lambda x: x.buffer(buffer_distance, cap_style=CAP_STYLE.flat))
    gdf_buffer['ref_idx'] = gdf_buffer.index  # Track which reference each buffer is from

    # Read the parquet file efficiently - only load id and geometry_wkt initially
    print("Reading parquet file (id, geometry_wkt fields only)...")
    parquet_fields = ['id', 'geometry_wkt']
    
    # Use pyarrow to read only selected columns
    try:
        parquet_table = pq.read_table(parquet_path, columns=parquet_fields)
        df_network = parquet_table.to_pandas()
        print(f"Successfully loaded {len(df_network)} rows")
    except Exception as e:
        print(f"Error reading parquet columns: {e}")
        print("Attempting to read all columns and then filter...")
        parquet_table = pq.read_table(parquet_path)
        df_network = parquet_table.to_pandas()
        # Keep only the columns we need
        required_cols = ['id', 'geometry_wkt']
        for col in required_cols:
            if col not in df_network.columns:
                raise ValueError(f"Required column '{col}' not found in parquet file.")
        df_network = df_network[required_cols]
        print(f"Successfully loaded {len(df_network)} rows")
    
    # Convert WKT to geometry objects
    print("Converting WKT to geometry objects...")
    df_network['geometry'] = df_network['geometry_wkt'].apply(lambda x: wkt.loads(x) if x else None)
    
    # Drop rows with invalid geometries
    df_network = df_network.dropna(subset=['geometry'])
    
    # Convert to GeoDataFrame
    gdf_network = gpd.GeoDataFrame(df_network, geometry='geometry')
    
    # Set CRS from the original GeoDataFrame
    gdf_network.set_crs(gdf_buffer.crs, inplace=True)
    
    # Create spatial index for more efficient intersection
    print("Creating spatial index...")
    sindex = gdf_network.sindex
    
    # Find potential intersections using the spatial index
    print("Finding potential intersections...")
    potential_matches_index = []
    
    for geom in gdf_buffer.geometry:
        potential_matches_index.extend(list(sindex.intersection(geom.bounds)))
    
    # Remove duplicates
    potential_matches_index = list(set(potential_matches_index))
    
    # Extract the candidates
    candidates = gdf_network.iloc[potential_matches_index].copy()
    
    # Perform actual intersection test
    print("Performing actual intersection test...")
    nodes_in_buffer = gpd.sjoin(
        candidates, 
        gdf_buffer, 
        how='inner', 
        predicate='intersects'
    )
    
    # Fix id column after spatial join (it becomes id_left)
    print("Checking id column after spatial join...")
    if 'id' not in nodes_in_buffer.columns and 'id_left' in nodes_in_buffer.columns:
        print("Renaming 'id_left' to 'id'")
        nodes_in_buffer['id'] = nodes_in_buffer['id_left']
    
    def calculate_direction_vector(geom):
        """
        Calculate the direction vector of a line segment by fitting a line to all its vertices.
        """
        if isinstance(geom, Point) or len(geom.coords) < 2:
            return None
        
        # Extract all vertices
        x = np.array([coord[0] for coord in geom.coords])
        y = np.array([coord[1] for coord in geom.coords])
        
        # Fit a line using linear regression
        model = LinearRegression()
        model.fit(x.reshape(-1, 1), y)
        
        # Calculate direction vector from the slope
        direction_vector = np.array([1, model.coef_[0]])  # [dx, dy]
        return direction_vector / np.linalg.norm(direction_vector)  # Normalize

    def calculate_parallel_score(geom, main_line):
        """
        Calculate how parallel a line segment is to the main street.
        """
        try:
            # Convert Point geometries to their closest main_line point
            if isinstance(geom, Point):
                return 0.0
                
            # Get direction vectors
            main_vector = calculate_direction_vector(main_line)
            segment_vector = calculate_direction_vector(geom)
            
            if main_vector is None or segment_vector is None:
                return 0.0
                
            # Calculate dot product and magnitudes
            dot_product = np.dot(main_vector, segment_vector)
            main_magnitude = np.linalg.norm(main_vector)
            segment_magnitude = np.linalg.norm(segment_vector)
            
            if main_magnitude == 0 or segment_magnitude == 0:
                return 0.0
                
            cos_angle = abs(dot_product / (main_magnitude * segment_magnitude))
            return float(cos_angle)
            
        except Exception as e:
            print(f"Error calculating parallel score: {e}")
            return 0.0

    # Process each reference geometry separately
    print("Processing each reference geometry...")
    all_parallel = []
    
    # Group by reference geometry
    for ref_idx, group in nodes_in_buffer.groupby('ref_idx'):
        print(f"Processing reference geometry {ref_idx+1}/{len(gdf)}", end='\r')
        
        # Get the reference geometry
        ref_geom = gdf.iloc[ref_idx].geometry
        
        # Calculate parallel scores
        group = group.copy()
        group['parallel_score'] = group.geometry.apply(
            lambda x: calculate_parallel_score(x, ref_geom)
        )
        
        # Filter parallel segments
        parallel_segments = group[group['parallel_score'] > parallel_threshold].copy()
        
        # Add to results
        if not parallel_segments.empty:
            # Sort by parallel score
            parallel_segments = parallel_segments.sort_values('parallel_score', ascending=False)
            all_parallel.append(parallel_segments)
    
    # Combine all parallel segments and remove duplicates
    if all_parallel:
        parallel_nodes = pd.concat(all_parallel, ignore_index=True)
        # Remove duplicates by ID (keeping the one with the highest parallel score)
        if 'id' in parallel_nodes.columns:
            parallel_nodes = parallel_nodes.sort_values('parallel_score', ascending=False)
            parallel_nodes = parallel_nodes.drop_duplicates(subset=['id'])
    else:
        parallel_nodes = gpd.GeoDataFrame(
            [], 
            geometry='geometry', 
            crs=gdf.crs,
            columns=nodes_in_buffer.columns if not nodes_in_buffer.empty else ['geometry']
        )

    print(f"Total nodes in buffer: {len(nodes_in_buffer)}")
    print(f"Parallel nodes found: {len(parallel_nodes)}")
    
    # Print some diagnostic information
    if len(parallel_nodes) > 0:
        print("\nParallel score statistics:")
        print(parallel_nodes['parallel_score'].describe())
    
    # If we want additional fields from the original parquet, re-join now with the filtered dataset
    if not parallel_nodes.empty and len(parallel_nodes) < 10000:  # Limit to reasonable size
        print("Loading additional fields for parallel segments...")
        # Get IDs of parallel segments
        # Verify id column exists
        if 'id' not in parallel_nodes.columns:
            print("ERROR: 'id' column not found, looking for alternatives...")
            if 'id_left' in parallel_nodes.columns:
                print("Using 'id_left' instead")
                parallel_ids = parallel_nodes['id_left'].tolist()
                id_field_to_use = 'id'  # The field in the original dataset is still called 'id'
            else:
                print("No suitable ID column found. Cannot load additional fields.")
                return nodes_in_buffer, parallel_nodes
        else:
            parallel_ids = parallel_nodes['id'].tolist()
            id_field_to_use = 'id'
            
        print(f"Found {len(parallel_ids)} unique IDs for rejoining")
        
        # Read only the needed rows using a filter expression
        # This is much more efficient than loading the entire file
        try:
            print(f"Reading additional fields for only {len(parallel_ids)} segments...")
            
            # For very large ID lists, we need to split the filter into chunks
            # to avoid exceeding filter expression size limits
            if len(parallel_ids) > 1000:
                print("Large number of IDs, splitting into chunks...")
                chunk_size = 1000
                chunks = [parallel_ids[i:i + chunk_size] for i in range(0, len(parallel_ids), chunk_size)]
                
                dfs = []
                for i, chunk in enumerate(chunks):
                    print(f"Reading chunk {i+1}/{len(chunks)}...", end='\r')
                    # Create filter expression for this chunk
                    filters = [('id', 'in', chunk)]
                    chunk_table = pq.read_table(
                        parquet_path,
                        filters=filters
                    )
                    dfs.append(chunk_table.to_pandas())
                
                # Combine all chunks
                df_full_filtered = pd.concat(dfs, ignore_index=True)
            else:
                # For smaller ID lists, we can do it in one go
                filters = [('id', 'in', parallel_ids)]
                table = pq.read_table(
                    parquet_path,
                    filters=filters
                )
                df_full_filtered = table.to_pandas()
                
            print(f"Successfully loaded {len(df_full_filtered)} rows with all fields")
            
        except Exception as e:
            print(f"Error using filters: {e}")
            print("Falling back to reading entire file and filtering...")
            
            # Fallback to reading the entire file
            parquet_table_full = pq.read_table(parquet_path)
            df_full = parquet_table_full.to_pandas()
            
            # Filter to only parallel segment IDs
            df_full_filtered = df_full[df_full[id_field_to_use].isin(parallel_ids)]
            print(f"Filtered to {len(df_full_filtered)} rows")
        
        # Get list of columns to merge (exclude geometry_wkt and any columns we already have)
        merge_columns = [col for col in df_full_filtered.columns 
                         if col != 'geometry_wkt' and col not in parallel_nodes.columns]
        
        if merge_columns:
            print(f"Merging {len(merge_columns)} additional fields: {', '.join(merge_columns[:5])}...")
            
            # Create a subset with only needed columns for more efficient merge
            merge_df = df_full_filtered[[id_field_to_use] + merge_columns].copy()
            
            # Merge with our parallel segments
            # Need to match the id field in parallel_nodes with id_field_to_use
            if 'id' in parallel_nodes.columns:
                parallel_nodes = parallel_nodes.merge(
                    merge_df, 
                    left_on='id',
                    right_on=id_field_to_use, 
                    how='left'
                )
            elif 'id_left' in parallel_nodes.columns:
                parallel_nodes = parallel_nodes.merge(
                    merge_df, 
                    left_on='id_left',
                    right_on=id_field_to_use, 
                    how='left'
                )
        else:
            print("No additional fields to merge.")
    else:
        if parallel_nodes.empty:
            print("No parallel segments found, skipping data re-join.")
        else:
            print(f"Too many parallel segments ({len(parallel_nodes)}) for full data re-join.")

    return nodes_in_buffer, parallel_nodes

def main():
    # Assumes gdf is already defined in the notebook
    nodes_buffer, gdf3 = find_parallel_segments(
        gdf, 
        'data/network_all_months_plus_25833_length_with_fahrradstrasse.parquet',
        # Buffer distance in meters
        buffer_distance=4,
        parallel_threshold=0.8
        # Filter parallel segments using the score column, change as needed:
        # 1.0 = completely parallel (0° angle)
        # 0.866 = 30° angle
        # 0.7 = approximately 45° angle
        # 0.5 = 60° angle
        # 0.0 = perpendicular (90° angle)
    )
    
    print(f"Created gdf3 with {len(gdf3)} parallel segments")
    
    return nodes_buffer, gdf3

if __name__ == "__main__":
    nodes_buffer, gdf3 = main()

Reading parquet file (id, geometry_wkt fields only)...
Successfully loaded 592136 rows
Converting WKT to geometry objects...
Creating spatial index...
Finding potential intersections...
Performing actual intersection test...
Checking id column after spatial join...
Renaming 'id_left' to 'id'
Processing each reference geometry...
Total nodes in buffer: 150try 25/25
Parallel nodes found: 61

Parallel score statistics:
count    61.000000
mean      0.992664
std       0.024564
min       0.842145
25%       0.998488
50%       0.999943
75%       0.999989
max       1.000000
Name: parallel_score, dtype: float64
Loading additional fields for parallel segments...
Found 61 unique IDs for rejoining
Reading additional fields for only 61 segments...
Successfully loaded 61 rows with all fields
Merging 803 additional fields: city_id, osmid, access, 2304-2412_speed, 2304-2412_route_count...
Created gdf3 with 61 parallel segments


In [3]:
# Step 2 v2.3 final var RADWEGE [FULL] [parquet]: Find parallel segments to the overpass data in RADWEGE DATA
# ADJUST buffer_distance and parallel_threshold at the end of the code!

import geopandas as gpd
import pandas as pd
import pyogrio
import numpy as np
from shapely.geometry import LineString, Point
from shapely import wkt
import pyarrow.parquet as pq
from sklearn.linear_model import LinearRegression  # For fitting a line to vertices
from shapely.geometry import CAP_STYLE

def find_parallel_segments(gdf, parquet_path, buffer_distance=15, parallel_threshold=0.8):
    """
    Find parallel segments from a parquet file with WKT geometries within a buffer of the input geometry.
    
    Parameters:
    -----------
    gdf : GeoDataFrame
        Input GeoDataFrame containing the reference street geometries (selection tool)
    parquet_path : str
        Path to the parquet file with WKT geometries (data source)
    buffer_distance : float
        Buffer distance in meters (default: 15)
    parallel_threshold : float
        Threshold for parallel score (default: 0.8)
        
    Returns:
    --------
    tuple : (nodes_in_buffer, parallel_nodes)
        Two GeoDataFrames containing all nodes in buffer and parallel nodes
    """
    
    # Create buffer for each reference geometry
    gdf_buffer = gdf.copy()
    # gdf_buffer['geometry'] = gdf.geometry.buffer(buffer_distance)
    gdf_buffer['geometry'] = gdf.geometry.apply(lambda x: x.buffer(buffer_distance, cap_style=CAP_STYLE.flat))
    gdf_buffer['ref_idx'] = gdf_buffer.index  # Track which reference each buffer is from

    # Read the parquet file efficiently - only load id and geometry_wkt initially
    print("Reading parquet file (id, geometry_wkt fields only)...")
    parquet_fields = ['id', 'geometry_wkt']
    
    # Use pyarrow to read only selected columns
    try:
        parquet_table = pq.read_table(parquet_path, columns=parquet_fields)
        df_network = parquet_table.to_pandas()
        print(f"Successfully loaded {len(df_network)} rows")
    except Exception as e:
        print(f"Error reading parquet columns: {e}")
        print("Attempting to read all columns and then filter...")
        parquet_table = pq.read_table(parquet_path)
        df_network = parquet_table.to_pandas()
        # Keep only the columns we need
        required_cols = ['id', 'geometry_wkt']
        for col in required_cols:
            if col not in df_network.columns:
                raise ValueError(f"Required column '{col}' not found in parquet file.")
        df_network = df_network[required_cols]
        print(f"Successfully loaded {len(df_network)} rows")
    
    # Convert WKT to geometry objects
    print("Converting WKT to geometry objects...")
    df_network['geometry'] = df_network['geometry_wkt'].apply(lambda x: wkt.loads(x) if x else None)
    
    # Drop rows with invalid geometries
    df_network = df_network.dropna(subset=['geometry'])
    
    # Convert to GeoDataFrame
    gdf_network = gpd.GeoDataFrame(df_network, geometry='geometry')
    
    # Set CRS from the original GeoDataFrame
    gdf_network.set_crs(gdf_buffer.crs, inplace=True)
    
    # Create spatial index for more efficient intersection
    print("Creating spatial index...")
    sindex = gdf_network.sindex
    
    # Find potential intersections using the spatial index
    print("Finding potential intersections...")
    potential_matches_index = []
    
    for geom in gdf_buffer.geometry:
        potential_matches_index.extend(list(sindex.intersection(geom.bounds)))
    
    # Remove duplicates
    potential_matches_index = list(set(potential_matches_index))
    
    # Extract the candidates
    candidates = gdf_network.iloc[potential_matches_index].copy()
    
    # Perform actual intersection test
    print("Performing actual intersection test...")
    nodes_in_buffer = gpd.sjoin(
        candidates, 
        gdf_buffer, 
        how='inner', 
        predicate='intersects'
    )
    
    # Fix id column after spatial join (it becomes id_left)
    print("Checking id column after spatial join...")
    if 'id' not in nodes_in_buffer.columns and 'id_left' in nodes_in_buffer.columns:
        print("Renaming 'id_left' to 'id'")
        nodes_in_buffer['id'] = nodes_in_buffer['id_left']
    
    def calculate_direction_vector(geom):
        """
        Calculate the direction vector of a line segment by fitting a line to all its vertices.
        """
        if isinstance(geom, Point) or len(geom.coords) < 2:
            return None
        
        # Extract all vertices
        x = np.array([coord[0] for coord in geom.coords])
        y = np.array([coord[1] for coord in geom.coords])
        
        # Fit a line using linear regression
        model = LinearRegression()
        model.fit(x.reshape(-1, 1), y)
        
        # Calculate direction vector from the slope
        direction_vector = np.array([1, model.coef_[0]])  # [dx, dy]
        return direction_vector / np.linalg.norm(direction_vector)  # Normalize

    def calculate_parallel_score(geom, main_line):
        """
        Calculate how parallel a line segment is to the main street.
        """
        try:
            # Convert Point geometries to their closest main_line point
            if isinstance(geom, Point):
                return 0.0
                
            # Get direction vectors
            main_vector = calculate_direction_vector(main_line)
            segment_vector = calculate_direction_vector(geom)
            
            if main_vector is None or segment_vector is None:
                return 0.0
                
            # Calculate dot product and magnitudes
            dot_product = np.dot(main_vector, segment_vector)
            main_magnitude = np.linalg.norm(main_vector)
            segment_magnitude = np.linalg.norm(segment_vector)
            
            if main_magnitude == 0 or segment_magnitude == 0:
                return 0.0
                
            cos_angle = abs(dot_product / (main_magnitude * segment_magnitude))
            return float(cos_angle)
            
        except Exception as e:
            print(f"Error calculating parallel score: {e}")
            return 0.0

    # Process each reference geometry separately
    print("Processing each reference geometry...")
    all_parallel = []
    
    # Group by reference geometry
    for ref_idx, group in nodes_in_buffer.groupby('ref_idx'):
        print(f"Processing reference geometry {ref_idx+1}/{len(gdf)}", end='\r')
        
        # Get the reference geometry
        ref_geom = gdf.iloc[ref_idx].geometry
        
        # Calculate parallel scores
        group = group.copy()
        group['parallel_score'] = group.geometry.apply(
            lambda x: calculate_parallel_score(x, ref_geom)
        )
        
        # Filter parallel segments
        parallel_segments = group[group['parallel_score'] > parallel_threshold].copy()
        
        # Add to results
        if not parallel_segments.empty:
            # Sort by parallel score
            parallel_segments = parallel_segments.sort_values('parallel_score', ascending=False)
            all_parallel.append(parallel_segments)
    
    # Combine all parallel segments and remove duplicates
    if all_parallel:
        parallel_nodes = pd.concat(all_parallel, ignore_index=True)
        # Remove duplicates by ID (keeping the one with the highest parallel score)
        if 'id' in parallel_nodes.columns:
            parallel_nodes = parallel_nodes.sort_values('parallel_score', ascending=False)
            parallel_nodes = parallel_nodes.drop_duplicates(subset=['id'])
    else:
        parallel_nodes = gpd.GeoDataFrame(
            [], 
            geometry='geometry', 
            crs=gdf.crs,
            columns=nodes_in_buffer.columns if not nodes_in_buffer.empty else ['geometry']
        )

    print(f"Total nodes in buffer: {len(nodes_in_buffer)}")
    print(f"Parallel nodes found: {len(parallel_nodes)}")
    
    # Print some diagnostic information
    if len(parallel_nodes) > 0:
        print("\nParallel score statistics:")
        print(parallel_nodes['parallel_score'].describe())
    
    # If we want additional fields from the original parquet, re-join now with the filtered dataset
    if not parallel_nodes.empty and len(parallel_nodes) < 10000:  # Limit to reasonable size
        print("Loading additional fields for parallel segments...")
        # Get IDs of parallel segments
        # Verify id column exists
        if 'id' not in parallel_nodes.columns:
            print("ERROR: 'id' column not found, looking for alternatives...")
            if 'id_left' in parallel_nodes.columns:
                print("Using 'id_left' instead")
                parallel_ids = parallel_nodes['id_left'].tolist()
                id_field_to_use = 'id'  # The field in the original dataset is still called 'id'
            else:
                print("No suitable ID column found. Cannot load additional fields.")
                return nodes_in_buffer, parallel_nodes
        else:
            parallel_ids = parallel_nodes['id'].tolist()
            id_field_to_use = 'id'
            
        print(f"Found {len(parallel_ids)} unique IDs for rejoining")
        
        # Read only the needed rows using a filter expression
        # This is much more efficient than loading the entire file
        try:
            print(f"Reading additional fields for only {len(parallel_ids)} segments...")
            
            # For very large ID lists, we need to split the filter into chunks
            # to avoid exceeding filter expression size limits
            if len(parallel_ids) > 1000:
                print("Large number of IDs, splitting into chunks...")
                chunk_size = 1000
                chunks = [parallel_ids[i:i + chunk_size] for i in range(0, len(parallel_ids), chunk_size)]
                
                dfs = []
                for i, chunk in enumerate(chunks):
                    print(f"Reading chunk {i+1}/{len(chunks)}...", end='\r')
                    # Create filter expression for this chunk
                    filters = [('id', 'in', chunk)]
                    chunk_table = pq.read_table(
                        parquet_path,
                        filters=filters
                    )
                    dfs.append(chunk_table.to_pandas())
                
                # Combine all chunks
                df_full_filtered = pd.concat(dfs, ignore_index=True)
            else:
                # For smaller ID lists, we can do it in one go
                filters = [('id', 'in', parallel_ids)]
                table = pq.read_table(
                    parquet_path,
                    filters=filters
                )
                df_full_filtered = table.to_pandas()
                
            print(f"Successfully loaded {len(df_full_filtered)} rows with all fields")
            
        except Exception as e:
            print(f"Error using filters: {e}")
            print("Falling back to reading entire file and filtering...")
            
            # Fallback to reading the entire file
            parquet_table_full = pq.read_table(parquet_path)
            df_full = parquet_table_full.to_pandas()
            
            # Filter to only parallel segment IDs
            df_full_filtered = df_full[df_full[id_field_to_use].isin(parallel_ids)]
            print(f"Filtered to {len(df_full_filtered)} rows")
        
        # Get list of columns to merge (exclude geometry_wkt and any columns we already have)
        merge_columns = [col for col in df_full_filtered.columns 
                         if col != 'geometry_wkt' and col not in parallel_nodes.columns]
        
        if merge_columns:
            print(f"Merging {len(merge_columns)} additional fields: {', '.join(merge_columns[:5])}...")
            
            # Create a subset with only needed columns for more efficient merge
            merge_df = df_full_filtered[[id_field_to_use] + merge_columns].copy()
            
            # Merge with our parallel segments
            # Need to match the id field in parallel_nodes with id_field_to_use
            if 'id' in parallel_nodes.columns:
                parallel_nodes = parallel_nodes.merge(
                    merge_df, 
                    left_on='id',
                    right_on=id_field_to_use, 
                    how='left'
                )
            elif 'id_left' in parallel_nodes.columns:
                parallel_nodes = parallel_nodes.merge(
                    merge_df, 
                    left_on='id_left',
                    right_on=id_field_to_use, 
                    how='left'
                )
        else:
            print("No additional fields to merge.")
    else:
        if parallel_nodes.empty:
            print("No parallel segments found, skipping data re-join.")
        else:
            print(f"Too many parallel segments ({len(parallel_nodes)}) for full data re-join.")

    return nodes_in_buffer, parallel_nodes

def main():
    # Assumes gdf is already defined in the notebook
    nodes_buffer, gdf3 = find_parallel_segments(
        gdf, 
        'data/network_all_months_plus_25833_RVA_clipped.parquet',
        # Buffer distance in meters
        buffer_distance=10,
        parallel_threshold=0.8
        # Filter parallel segments using the score column, change as needed:
        # 1.0 = completely parallel (0° angle)
        # 0.866 = 30° angle
        # 0.7 = approximately 45° angle
        # 0.5 = 60° angle
        # 0.0 = perpendicular (90° angle)
    )
    
    print(f"Created gdf3 with {len(gdf3)} parallel segments")
    
    return nodes_buffer, gdf3

if __name__ == "__main__":
    nodes_buffer, gdf3 = main()

Reading parquet file (id, geometry_wkt fields only)...
Successfully loaded 140632 rows
Converting WKT to geometry objects...
Creating spatial index...
Finding potential intersections...
Performing actual intersection test...
Checking id column after spatial join...
Renaming 'id_left' to 'id'
Processing each reference geometry...
Total nodes in buffer: 652try 148/158
Parallel nodes found: 290

Parallel score statistics:
count    290.000000
mean       0.987946
std        0.029542
min        0.837761
25%        0.996547
50%        0.999774
75%        0.999987
max        1.000000
Name: parallel_score, dtype: float64
Loading additional fields for parallel segments...
Found 290 unique IDs for rejoining
Reading additional fields for only 290 segments...
Successfully loaded 436 rows with all fields
Merging 814 additional fields: sobj_kz, segm_segm, segm_bez, stst_str, stor_name...
Created gdf3 with 436 parallel segments


In [3]:
# Step 2 v2.3 final var HELPER-PARQUET [FULL] [parquet]: Find parallel and mark IDs with new field = 1

import geopandas as gpd
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
from shapely.geometry import LineString, Point
from shapely import wkt
from sklearn.linear_model import LinearRegression
from shapely.geometry import CAP_STYLE
import os
import time

# Configuration parameters
CONFIG = {
    'parquet_path': 'data/network_all_months_plus_25833_length_with_fahrradstrasse.parquet',
    'helper_parquet_path': 'data/helper_segments.parquet',
    'buffer_distance': 5,  # Buffer distance in meters
    'parallel_threshold': 0.8,  # Threshold for parallel score
    'indicator_field_name': 'protected_bike_lane'  # Change this for each new analysis
}

def find_parallel_segments(gdf, config=CONFIG):
    """
    Find parallel segments from a parquet file within a buffer distance of the input geometry.
    Properly updates the helper parquet file with a new indicator field.
    
    Parameters:
    -----------
    gdf : GeoDataFrame
        Input GeoDataFrame containing the reference street geometries
    config : dict
        Configuration dictionary with parameters
        
    Returns:
    --------
    parallel_nodes_simple : GeoDataFrame
        GeoDataFrame containing the parallel segments with basic information
    """
    start_time = time.time()
    print(f"Starting process with buffer distance: {config['buffer_distance']}m and parallel threshold: {config['parallel_threshold']}")
    
    # Create buffer for each reference geometry
    gdf_buffer = gdf.copy()
    gdf_buffer['geometry'] = gdf.geometry.apply(lambda x: x.buffer(config['buffer_distance'], cap_style=CAP_STYLE.flat))
    gdf_buffer['ref_idx'] = gdf_buffer.index
    
    # Read the parquet file - only load id and geometry_wkt initially
    print(f"Reading parquet file (id, geometry_wkt fields only)...")
    parquet_fields = ['id', 'geometry_wkt']
    
    try:
        parquet_table = pq.read_table(config['parquet_path'], columns=parquet_fields)
        df_network = parquet_table.to_pandas()
        print(f"Successfully loaded {len(df_network)} rows with minimal fields")
    except Exception as e:
        print(f"Error reading parquet columns: {e}")
        print("Attempting to read all columns and then filter...")
        parquet_table = pq.read_table(config['parquet_path'])
        df_network = parquet_table.to_pandas()
        required_cols = ['id', 'geometry_wkt']
        for col in required_cols:
            if col not in df_network.columns:
                raise ValueError(f"Required column '{col}' not found in parquet file.")
        df_network = df_network[required_cols]
        print(f"Successfully loaded {len(df_network)} rows, filtered to minimal fields")
    
    # Convert WKT to geometry objects
    print("Converting WKT to geometry objects...")
    df_network['geometry'] = df_network['geometry_wkt'].apply(lambda x: wkt.loads(x) if x else None)
    df_network = df_network.dropna(subset=['geometry'])
    
    # Convert to GeoDataFrame
    gdf_network = gpd.GeoDataFrame(df_network, geometry='geometry')
    gdf_network.set_crs(gdf_buffer.crs, inplace=True)
    
    # Create spatial index and find potential intersections
    print("Creating spatial index and finding intersections...")
    sindex = gdf_network.sindex
    potential_matches_index = []
    
    for geom in gdf_buffer.geometry:
        potential_matches_index.extend(list(sindex.intersection(geom.bounds)))
    
    potential_matches_index = list(set(potential_matches_index))
    candidates = gdf_network.iloc[potential_matches_index].copy()
    
    # Perform spatial join
    print("Performing spatial join...")
    nodes_in_buffer = gpd.sjoin(
        candidates, 
        gdf_buffer, 
        how='inner', 
        predicate='intersects'
    )
    
    # Fix id column if needed
    if 'id' not in nodes_in_buffer.columns and 'id_left' in nodes_in_buffer.columns:
        print("Renaming 'id_left' to 'id'")
        nodes_in_buffer['id'] = nodes_in_buffer['id_left']
    
    # Direction vector calculation function
    def calculate_direction_vector(geom):
        if isinstance(geom, Point) or len(geom.coords) < 2:
            return None
        
        x = np.array([coord[0] for coord in geom.coords])
        y = np.array([coord[1] for coord in geom.coords])
        
        model = LinearRegression()
        model.fit(x.reshape(-1, 1), y)
        
        direction_vector = np.array([1, model.coef_[0]])
        return direction_vector / np.linalg.norm(direction_vector)

    # Parallel score calculation function
    def calculate_parallel_score(geom, main_line):
        try:
            if isinstance(geom, Point):
                return 0.0
                
            main_vector = calculate_direction_vector(main_line)
            segment_vector = calculate_direction_vector(geom)
            
            if main_vector is None or segment_vector is None:
                return 0.0
                
            dot_product = np.dot(main_vector, segment_vector)
            cos_angle = abs(dot_product)
            return float(cos_angle)
            
        except Exception as e:
            print(f"Error calculating parallel score: {e}")
            return 0.0

    # Process each reference geometry to find parallel segments
    print("Processing each reference geometry...")
    all_parallel = []
    
    for ref_idx, group in nodes_in_buffer.groupby('ref_idx'):
        print(f"Processing reference geometry {ref_idx+1}/{len(gdf)}", end='\r')
        
        ref_geom = gdf.iloc[ref_idx].geometry
        
        group = group.copy()
        group['parallel_score'] = group.geometry.apply(
            lambda x: calculate_parallel_score(x, ref_geom)
        )
        
        parallel_segments = group[group['parallel_score'] > config['parallel_threshold']].copy()
        
        if not parallel_segments.empty:
            parallel_segments = parallel_segments.sort_values('parallel_score', ascending=False)
            all_parallel.append(parallel_segments)
    
    # Combine all parallel segments
    if all_parallel:
        parallel_nodes = pd.concat(all_parallel, ignore_index=True)
        parallel_nodes = parallel_nodes.sort_values('parallel_score', ascending=False)
        parallel_nodes = parallel_nodes.drop_duplicates(subset=['id'])
    else:
        parallel_nodes = gpd.GeoDataFrame(
            [], 
            geometry='geometry', 
            crs=gdf.crs,
            columns=nodes_in_buffer.columns if not nodes_in_buffer.empty else ['geometry']
        )

    print(f"\nTotal nodes in buffer: {len(nodes_in_buffer)}")
    print(f"Parallel nodes found: {len(parallel_nodes)}")
    
    # Extract the IDs of parallel segments
    if 'id' in parallel_nodes.columns:
        parallel_ids = set(parallel_nodes['id'].tolist())
    elif 'id_left' in parallel_nodes.columns:
        parallel_ids = set(parallel_nodes['id_left'].tolist())
    else:
        print("ERROR: No ID column found in parallel nodes.")
        return parallel_nodes
    
    print(f"Found {len(parallel_ids)} unique IDs for parallel segments")
    
    # Create a simplified version of parallel_nodes with just the essential info
    parallel_nodes_simple = parallel_nodes[['id', 'geometry', 'parallel_score']].copy()
    
    # =========================================================================
    # THIS IS THE FIXED PART: Properly update the helper parquet file
    # =========================================================================
    
    def properly_update_helper_parquet():
        """
        Properly update the helper parquet file without losing existing data.
        """
        output_path = config['helper_parquet_path']
        field_name = config['indicator_field_name']
        score_field = f"{field_name}_score"
        
        # Check if output directory exists
        output_dir = os.path.dirname(output_path)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # Check if helper file already exists
        helper_exists = os.path.exists(output_path)
        
        if helper_exists:
            print(f"Updating existing helper file at {output_path}")
            # Read existing helper file
            helper_df = pd.read_parquet(output_path)
            original_row_count = len(helper_df)
            print(f"Existing helper file has {original_row_count} rows")
            
            # Get existing IDs in the helper file
            existing_ids = set(helper_df['id'].tolist())
            
            # Find IDs that exist in parallel_ids but not in existing_ids
            new_ids = parallel_ids - existing_ids
            print(f"Found {len(new_ids)} new IDs to add to helper file")
            
            # Add new column to existing helper file (with default value 0)
            if field_name not in helper_df.columns:
                helper_df[field_name] = 0
            
            # Update values for existing rows
            helper_df.loc[helper_df['id'].isin(parallel_ids), field_name] = 1
            
            # Add score field if we have scores
            if len(parallel_nodes) > 0 and 'parallel_score' in parallel_nodes.columns:
                # Create mapping of ID to score
                id_to_score = {row['id']: row['parallel_score'] 
                              for _, row in parallel_nodes.iterrows() 
                              if 'id' in row and 'parallel_score' in row}
                
                # Add score column
                if score_field not in helper_df.columns:
                    helper_df[score_field] = np.nan
                
                # Update scores for existing rows
                for id_val, score in id_to_score.items():
                    helper_df.loc[helper_df['id'] == id_val, score_field] = score
            
            # Create DataFrame for new rows that don't exist yet
            if new_ids:
                new_rows = []
                
                for id_val in new_ids:
                    # Create a dictionary for the new row with default values
                    row_dict = {'id': id_val}
                    
                    # Set 0 for all existing indicator columns
                    for col in helper_df.columns:
                        if col != 'id' and not col.endswith('_score'):
                            row_dict[col] = 0
                    
                    # Set 1 for the current indicator
                    row_dict[field_name] = 1
                    
                    # Add score if available
                    if len(parallel_nodes) > 0 and 'parallel_score' in parallel_nodes.columns:
                        score_rows = parallel_nodes[parallel_nodes['id'] == id_val]
                        if len(score_rows) > 0:
                            row_dict[score_field] = score_rows.iloc[0]['parallel_score']
                    
                    new_rows.append(row_dict)
                
                # Create DataFrame and append to existing
                if new_rows:
                    new_df = pd.DataFrame(new_rows)
                    helper_df = pd.concat([helper_df, new_df], ignore_index=True)
            
            # Write back to parquet
            helper_df.to_parquet(output_path, index=False)
            
            print(f"Updated helper file now has {len(helper_df)} rows (added {len(helper_df) - original_row_count} new rows)")
            print(f"Total segments with {field_name}=1: {helper_df[field_name].sum()}")
            
        else:
            print(f"Creating new helper file at {output_path}")
            
            # For a new file, we only need to include the parallel IDs
            rows = []
            
            for id_val in parallel_ids:
                row_dict = {
                    'id': id_val,
                    field_name: 1
                }
                
                # Add score if available
                #if len(parallel_nodes) > 0 and 'parallel_score' in parallel_nodes.columns:
                    #score_rows = parallel_nodes[parallel_nodes['id'] == id_val]
                    #if len(score_rows) > 0:
                        #row_dict[score_field] = score_rows.iloc[0]['parallel_score']
                
                rows.append(row_dict)
            
            # Create DataFrame
            helper_df = pd.DataFrame(rows)
            
            # Write to parquet
            helper_df.to_parquet(output_path, index=False)
            
            print(f"Created new helper file with {len(helper_df)} rows")
        
        # Calculate percentage of GDF IDs with matches
        total_gdf_ids = len(gdf)
        matching_percentage = (len(parallel_ids) / total_gdf_ids) * 100 if total_gdf_ids > 0 else 0
        print(f"Percentage of GDF IDs with matching parquet ID: {matching_percentage:.2f}%")
        
        return len(parallel_ids), total_gdf_ids
    
    # Call the fixed function to update helper parquet
    total_parallel, total_gdf = properly_update_helper_parquet()
    
    end_time = time.time()
    processing_time = end_time - start_time
    print(f"Total processing time: {processing_time:.2f} seconds ({processing_time/60:.2f} minutes)")
    
    return parallel_nodes_simple

def main():
    # For command-line usage
    config = CONFIG.copy()
    
    # Load your GeoDataFrame here or assume it's already defined in the notebook
    # gdf = gpd.read_file('path_to_your_gdf.gpkg')
    
    print("Configuration:")
    for key, value in config.items():
        print(f"  {key}: {value}")
    
    parallel_segments = find_parallel_segments(gdf, config)
    
    print(f"Created simplified parallel segments GeoDataFrame with {len(parallel_segments)} rows")
    print(f"Added field '{config['indicator_field_name']}' to helper parquet at {config['helper_parquet_path']}")
    
    return parallel_segments

if __name__ == "__main__":
    parallel_segments = main()

Configuration:
  parquet_path: data/network_all_months_plus_25833_length_with_fahrradstrasse.parquet
  helper_parquet_path: data/helper_segments.parquet
  buffer_distance: 5
  parallel_threshold: 0.8
  indicator_field_name: protected_bike_lane
Starting process with buffer distance: 5m and parallel threshold: 0.8
Reading parquet file (id, geometry_wkt fields only)...
Successfully loaded 592136 rows with minimal fields
Converting WKT to geometry objects...
Creating spatial index and finding intersections...
Performing spatial join...
Renaming 'id_left' to 'id'
Processing each reference geometry...
Processing reference geometry 158/158
Total nodes in buffer: 1215
Parallel nodes found: 615
Found 615 unique IDs for parallel segments
Creating new helper file at data/helper_segments.parquet
Created new helper file with 615 rows
Percentage of GDF IDs with matching parquet ID: 389.24%
Total processing time: 11.59 seconds (0.19 minutes)
Created simplified parallel segments GeoDataFrame with 615 

In [3]:
# 2.1: Check if necessary
# gdf3.explore()

# Create a simplified GeoDataFrame with only id and geometry
gdf3_simple = gdf3[['id', 'geometry']].copy()

# Create the interactive map
gdf3_simple.explore()

In [3]:
# Step 3 v4.2: [FULL] Analyze values by month interactively + processing statistical values on the fly

import geopandas as gpd
import pandas as pd
import numpy as np
import ipyleaflet
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import json
import warnings
from shapely.geometry import Point, shape
from scipy.stats import entropy
from typing import Optional, Tuple
from IPython.display import display
import ipywidgets as widgets

# Speed Statistics Calculator Class
class SpeedStatistics:
    """Class to handle speed histogram statistics calculations"""
    
    def __init__(self, n_bins: int = 32):
        self.n_bins = n_bins
        self.bin_edges = np.arange(0, n_bins + 1)
        self.midpoints = (self.bin_edges[:-1] + self.bin_edges[1:]) / 2
    
    def parse_histogram(self, binned_speeds) -> Optional[np.ndarray]:
        """Safely parse histogram string to numpy array"""
        if binned_speeds is None:
            return None
        
        if isinstance(binned_speeds, (list, np.ndarray)):
            # If already a list or array, convert to numpy array
            return np.array(binned_speeds)
            
        if pd.isna(binned_speeds):
            return None
            
        try:
            if isinstance(binned_speeds, str):
                hist = eval(binned_speeds)
            else:
                hist = binned_speeds
                
            hist_array = np.array(hist)
            if len(hist_array) != self.n_bins:
                warnings.warn(f"Expected {self.n_bins} bins, got {len(hist_array)}")
                return None
            return hist_array
            
        except Exception as e:
            warnings.warn(f"Error parsing histogram: {e}")
            return None
    
    def calculate_stats(self, binned_speeds) -> Tuple:
        """Calculate comprehensive statistics for a speed histogram"""
        # Parse histogram
        hist_array = self.parse_histogram(binned_speeds)
        if hist_array is None:
            return (np.nan,) * 15
        
        total_count = hist_array.sum()
        if total_count == 0:
            return (np.nan,) * 15
            
        # Calculate probabilities
        probs = hist_array / total_count
        
        # Basic statistics
        weighted_mean = np.sum(self.midpoints * probs)
        weighted_var = np.sum(((self.midpoints - weighted_mean) ** 2) * probs)
        weighted_std = np.sqrt(weighted_var)
        
        # Higher moments
        if weighted_std > 0:
            standardized_moments = (self.midpoints - weighted_mean) / weighted_std
            skewness = np.sum((standardized_moments ** 3) * probs)
            kurtosis = np.sum((standardized_moments ** 4) * probs) - 3
        else:
            skewness = np.nan
            kurtosis = np.nan
        
        # Calculate CV
        cv = (weighted_std / weighted_mean * 100) if weighted_mean > 0 else np.nan
        
        # Entropy (only for non-zero probabilities)
        valid_probs = probs[probs > 0]
        data_entropy = entropy(valid_probs, base=2) if len(valid_probs) > 0 else np.nan
        
        # Quantiles
        cum_probs = np.cumsum(hist_array) / total_count
        q1 = np.interp(0.25, cum_probs, self.midpoints)
        q2 = np.interp(0.50, cum_probs, self.midpoints)
        q3 = np.interp(0.75, cum_probs, self.midpoints)
        q4 = np.interp(1.00, cum_probs, self.midpoints)
        iqr = q3 - q1
        
        # Speed range percentages
        pct_0_5 = np.interp(5, self.midpoints, cum_probs, left=0) * 100
        pct_5_15 = (np.interp(15, self.midpoints, cum_probs) - 
                   np.interp(5, self.midpoints, cum_probs)) * 100
        pct_15_25 = (np.interp(25, self.midpoints, cum_probs) - 
                    np.interp(15, self.midpoints, cum_probs)) * 100
        pct_25_plus = (1 - np.interp(25, self.midpoints, cum_probs, right=1)) * 100
        
        return tuple(round(x, 2) if not pd.isna(x) else x for x in [
            weighted_mean, weighted_std, cv, skewness, kurtosis,
            data_entropy, iqr, q1, q2, q3, q4,
            pct_0_5, pct_5_15, pct_15_25, pct_25_plus
        ])

# Get available metrics and times for analysis
def get_available_metrics_and_times(gdf):
    """Get available metrics and time periods from the data"""
    # Find all columns that match the pattern YY-MM_metric
    time_cols = sorted([col for col in gdf.columns 
                        if len(col.split('-')) > 1 and 
                        len(col.split('-')[0]) == 2 and 
                        len(col.split('-')[1].split('_')[0]) == 2])
    
    metrics = set()
    time_periods = set()
    
    for col in time_cols:
        parts = col.split('_', 1)
        if len(parts) > 1:
            yy_mm = parts[0]
            metric = parts[1]
            
            yy, mm = yy_mm.split('-')
            time_periods.add(f"20{yy}-{mm}")
            metrics.add(metric)
    
    # Define core metrics (even if they don't exist in the data)
    # We'll calculate these on-the-fly from speeds histograms if needed
    core_metrics = {'speeds', 'speed', 'route_count', 'cv', 'ent', 'skew', 'kurt', 'std', 'iqr', 
                    'q1', 'q2', 'q3', 'q4', 'p_0_5', 'p_5_15', 'p_15_25', 'p_25p'}
    
    # Only include metrics that have corresponding speed or speeds histograms
    available_metrics = {'route_count'}
    for period in time_periods:
        yy = period.split('-')[0][-2:]
        mm = period.split('-')[1]
        if f"{yy}-{mm}_speed" in gdf.columns:
            available_metrics.add('speed')
        if f"{yy}-{mm}_speeds" in gdf.columns:
            available_metrics.add('speeds')
        if f"{yy}-{mm}_cv" in gdf.columns:
            available_metrics.add('cv')
        if f"{yy}-{mm}_ent" in gdf.columns:
            available_metrics.add('ent')
        if f"{yy}-{mm}_skew" in gdf.columns:
            available_metrics.add('skew')
    
    # Add derived metrics that we can calculate if not already in the data
    derived_metrics = {'speed', 'cv', 'ent', 'skew', 'kurt', 'std', 'iqr', 
                      'q1', 'q2', 'q3', 'q4', 'p_0_5', 'p_5_15', 'p_15_25', 'p_25p'}
    
    # Merge available and derived metrics
    filtered_metrics = available_metrics.union(derived_metrics)
    
    return sorted(list(filtered_metrics)), sorted(list(time_periods))

# Process temporal data for analysis
def process_temporal_data(gdf, metric, selected_dates):
    """Process temporal data for given metric and dates"""
    if not metric or not selected_dates:
        return None
        
    # Initialize the speed statistics calculator
    calculator = SpeedStatistics()
    
    if metric == 'General Comparison':
        # Expanded metrics list to include quantiles and speed ranges
        metrics = ['speeds', 'cv', 'ent', 'skew', 'q1', 'q2', 'q3', 'iqr', 'p_0_5', 'p_5_15', 'p_15_25', 'p_25p']
        results = {}
        
        # First get all the pre-calculated speed values
        speed_values = []
        for date in selected_dates:
            year = date.split('-')[0][-2:]
            month = date.split('-')[1]
            speed_col = f"{year}-{month}_speed"
            if speed_col in gdf.columns:
                values = gdf[speed_col].dropna().tolist()
                speed_values.extend(values)
        
        if speed_values:
            results['speed'] = speed_values
            
        # For speeds histograms, sum them
        histograms = []
        for date in selected_dates:
            year = date.split('-')[0][-2:]
            month = date.split('-')[1]
            speeds_col = f"{year}-{month}_speeds"
            if speeds_col in gdf.columns:
                for hist_str in gdf[speeds_col]:
                    if pd.notna(hist_str):
                        hist = eval(hist_str) if isinstance(hist_str, str) else hist_str
                        histograms.append(hist)
        
        if histograms:
            results['speeds'] = np.sum(histograms, axis=0)
            
        # Metric index in stats tuple
        metric_index = {
            'cv': 2,       # coefficient of variation
            'skew': 3,     # skewness
            'ent': 5,      # entropy
            'iqr': 6,      # interquartile range
            'q1': 7,       # first quartile
            'q2': 8,       # median
            'q3': 9,       # third quartile
            'p_0_5': 11,   # % speeds 0-5 km/h
            'p_5_15': 12,  # % speeds 5-15 km/h
            'p_15_25': 13, # % speeds 15-25 km/h
            'p_25p': 14    # % speeds 25+ km/h
        }
        
        # Try using pre-calculated metrics first, then calculate from histograms if needed
        for m in metrics:
            if m == 'speeds':
                continue  # Already handled above
                
            values = []
            for date in selected_dates:
                year = date.split('-')[0][-2:]
                month = date.split('-')[1]
                metric_col = f"{year}-{month}_{m}"
                if metric_col in gdf.columns:
                    metric_values = gdf[metric_col].dropna().tolist()
                    values.extend(metric_values)
            
            if values:
                results[m] = values
            elif histograms and m in metric_index:
                # Calculate from histograms if we don't have pre-calculated values
                calculated_values = []
                for hist in histograms:
                    stats = calculator.calculate_stats(hist)
                    if not pd.isna(stats[metric_index[m]]):
                        calculated_values.append(stats[metric_index[m]])
                
                if calculated_values:
                    results[m] = calculated_values
        
        return results if results else None
    
    elif metric == 'speed':
        # Try to use pre-calculated speed values first
        values = []
        for date in selected_dates:
            year = date.split('-')[0][-2:]
            month = date.split('-')[1]
            col = f"{year}-{month}_speed"
            if col in gdf.columns:
                values.extend(gdf[col].dropna().tolist())
        
        if values:
            return values
        
        # If no pre-calculated values, calculate from histograms
        values = []
        for date in selected_dates:
            year = date.split('-')[0][-2:]
            month = date.split('-')[1]
            speeds_col = f"{year}-{month}_speeds"
            
            if speeds_col in gdf.columns:
                for hist_str in gdf[speeds_col]:
                    if pd.notna(hist_str):
                        stats = calculator.calculate_stats(hist_str)
                        if not pd.isna(stats[0]):  # Mean speed is the first value
                            values.append(stats[0])
        
        return values if values else None
    
    elif metric == 'speeds':
        # Sum all histograms for the speeds metric
        histograms = []
        for date in selected_dates:
            year = date.split('-')[0][-2:]
            month = date.split('-')[1]
            col = f"{year}-{month}_speeds"
            if col in gdf.columns:
                for hist_str in gdf[col]:
                    if pd.notna(hist_str):
                        hist = eval(hist_str) if isinstance(hist_str, str) else hist_str
                        histograms.append(hist)
        return np.sum(histograms, axis=0) if histograms else None
    
    elif metric in ['cv', 'ent', 'skew', 'kurt', 'std', 'iqr', 
                    'q1', 'q2', 'q3', 'q4', 'p_0_5', 'p_5_15', 'p_15_25', 'p_25p']:
        # First try to get pre-calculated values
        values = []
        for date in selected_dates:
            year = date.split('-')[0][-2:]
            month = date.split('-')[1]
            col = f"{year}-{month}_{metric}"
            if col in gdf.columns:
                values.extend(gdf[col].dropna().tolist())
        
        if values:
            return values
        
        # If no pre-calculated values, calculate from histograms
        metric_index = {
            'speed': 0,    # mean
            'std': 1,      # standard deviation
            'cv': 2,       # coefficient of variation  
            'skew': 3,     # skewness
            'kurt': 4,     # kurtosis
            'ent': 5,      # entropy
            'iqr': 6,      # interquartile range
            'q1': 7,       # first quartile
            'q2': 8,       # median
            'q3': 9,       # third quartile
            'q4': 10,      # maximum
            'p_0_5': 11,   # % speeds 0-5 km/h
            'p_5_15': 12,  # % speeds 5-15 km/h
            'p_15_25': 13, # % speeds 15-25 km/h
            'p_25p': 14    # % speeds 25+ km/h
        }
        
        index = metric_index.get(metric, 0)
        values = []
        
        for date in selected_dates:
            year = date.split('-')[0][-2:]
            month = date.split('-')[1]
            speeds_col = f"{year}-{month}_speeds"
            
            if speeds_col in gdf.columns:
                for hist_str in gdf[speeds_col]:
                    if pd.notna(hist_str):
                        stats = calculator.calculate_stats(hist_str)
                        if not pd.isna(stats[index]):
                            values.append(stats[index])
        
        return values if values else None
    
    else:
        # For other metrics like route_count that may already exist
        values = []
        for date in selected_dates:
            year = date.split('-')[0][-2:]
            month = date.split('-')[1]
            col = f"{year}-{month}_{metric}"
            if col in gdf.columns:
                values.extend(gdf[col].dropna().tolist())
        return values if values else None

# Create interactive map
def create_interactive_map(gdf, original_gdf=None):
    """Create interactive map with draw controls"""
    gdf_wgs84 = gdf.to_crs('EPSG:4326')
    center_lat = gdf_wgs84.total_bounds[[1,3]].mean()
    center_lon = gdf_wgs84.total_bounds[[0,2]].mean()
    
    m = ipyleaflet.Map(
        center=(center_lat, center_lon),
        zoom=14,
        scroll_wheel_zoom=True
    )
    
    draw_control = ipyleaflet.DrawControl(
        polygon={"shapeOptions": {"color": "#ff0000"}},
        rectangle={"shapeOptions": {"color": "#ff0000"}},
        circle={"shapeOptions": {"color": "#ff0000"}},
        circlemarker={},
        polyline={},
        marker={}
    )
    m.add_control(draw_control)
    
    # Style based on segment type if 'is_original' column exists
    if 'is_original' in gdf_wgs84.columns:
        # Speed field for coloring
        speed_field = '2304-2412_speed'
        if speed_field not in gdf_wgs84.columns:
            speed_fields = [col for col in gdf_wgs84.columns if col.endswith('_speed')]
            if speed_fields:
                speed_field = speed_fields[0]
            else:
                speed_field = None
        
        # Setup color normalization for speed values if available
        if speed_field:
            valid_speeds = gdf_wgs84[gdf_wgs84['is_original'] == False][speed_field].dropna()
            if len(valid_speeds) > 0:
                norm = plt.Normalize(valid_speeds.min(), valid_speeds.max())
            else:
                norm = plt.Normalize(0, 30)
            cmap = plt.cm.viridis
        
        def style_callback(feature):
            # Check if this is an original segment
            is_original = feature['properties'].get('is_original', False)
            
            if is_original:
                return {
                    'color': '#FF0000',  # Red for original segments
                    'weight': 5,
                    'opacity': 1.0,
                    'dashArray': '5,5'  # Dashed line for original segments
                }
            else:
                # For parallel segments, use speed-based coloring if available
                if speed_field and speed_field in feature['properties']:
                    speed = feature['properties'][speed_field]
                    if pd.notna(speed):
                        color = mcolors.rgb2hex(cmap(norm(speed)))
                    else:
                        color = '#808080'  # Gray for unknown speed
                else:
                    color = '#0000FF'  # Blue for parallel segments with no speed data
                    
                return {
                    'color': color,
                    'weight': 3,
                    'opacity': 0.7
                }
    else:
        # Use the standard speed-based styling
        # Use the new field name as specified
        speed_field = '2304-2412_speed'
        
        # If speed field doesn't exist, calculate it from the speed histogram
        if speed_field not in gdf_wgs84.columns and '2304-2412_speeds' in gdf_wgs84.columns:
            calculator = SpeedStatistics()
            gdf_wgs84[speed_field] = gdf_wgs84['2304-2412_speeds'].apply(
                lambda x: calculator.calculate_stats(x)[0] if pd.notna(x) else None
            )
        
        # Fallback options if still not available
        if speed_field not in gdf_wgs84.columns:
            # Try to find any existing speed field
            speed_fields = [col for col in gdf_wgs84.columns if col.endswith('_speed')]
            if speed_fields:
                speed_field = speed_fields[0]
            else:
                # Or calculate from any speeds histogram field
                speeds_fields = [col for col in gdf_wgs84.columns if col.endswith('_speeds')]
                if speeds_fields:
                    speed_field = speeds_fields[0].replace('_speeds', '_speed')
                    calculator = SpeedStatistics()
                    gdf_wgs84[speed_field] = gdf_wgs84[speeds_fields[0]].apply(
                        lambda x: calculator.calculate_stats(x)[0] if pd.notna(x) else None
                    )
                else:
                    # If nothing else, use a default column
                    speed_field = gdf_wgs84.columns[0]
        
        # Filter out NaN values for normalization
        valid_speeds = gdf_wgs84[speed_field].dropna()
        if len(valid_speeds) > 0:
            norm = plt.Normalize(valid_speeds.min(), valid_speeds.max())
        else:
            # Fallback if no valid speed values
            norm = plt.Normalize(0, 30)  # Reasonable default for km/h
        
        cmap = plt.cm.viridis

        def style_callback(feature):
            speed = feature['properties'][speed_field] if speed_field in feature['properties'] else None
            color = '#808080' if pd.isna(speed) else mcolors.rgb2hex(cmap(norm(speed)))
            return {'color': color, 'weight': 3, 'opacity': 0.7}

    geo_json = ipyleaflet.GeoJSON(
        data=json.loads(gdf_wgs84.to_json()),
        style={'weight': 3, 'opacity': 0.7},
        hover_style={'color': 'red', 'fillOpacity': 0.9},
        style_callback=style_callback
    )
    m.add_layer(geo_json)
    
    # Add the original geometries if provided and not already included
    if original_gdf is not None and 'is_original' not in gdf_wgs84.columns:
        # Convert to same CRS
        original_wgs84 = original_gdf.to_crs('EPSG:4326')
        
        # Add as a highlighted layer
        original_json = ipyleaflet.GeoJSON(
            data=json.loads(original_wgs84.to_json()),
            style={
                'color': '#FF0000',  # Red color
                'weight': 5,         # Thicker line
                'opacity': 1.0,      # Fully opaque
                'dashArray': '5,5'   # Dashed line pattern
            },
            hover_style={
                'color': '#FF00FF',  # Magenta on hover
                'weight': 7
            },
            name='Original Reference'
        )
        m.add_layer(original_json)
    
    return m, draw_control

# Create analysis widgets
def create_analysis_widgets(gdf, m, draw_control):
    """Create analysis widgets with comparison mode"""
    global ORIGINAL_GDF
    ORIGINAL_GDF = gdf.copy()
    
    metrics, time_periods = get_available_metrics_and_times(gdf)
    all_metrics = ['General Comparison'] + metrics
    
    # Create widgets
    comparison_mode = widgets.Checkbox(
        value=False,
        description='Comparison Mode',
        style={'description_width': 'initial'}
    )
    
    metric_selector = widgets.Dropdown(
        options=all_metrics,
        value=all_metrics[0],
        description='Metric:',
        style={'description_width': 'initial'}
    )
    
    time_selection1 = widgets.SelectMultiple(
        options=time_periods,
        description='Period 1:',
        style={'description_width': 'initial'},
        layout={'width': 'auto'}
    )
    
    time_selection2 = widgets.Box([
        widgets.SelectMultiple(
            options=time_periods,
            description='Period 2:',
            style={'description_width': 'initial'},
            layout={'width': 'auto'}
        )
    ], layout=widgets.Layout(display='none'))
    
    # Add segment type filter if 'is_original' column exists
    if 'is_original' in gdf.columns:
        segment_filter = widgets.Dropdown(
            options=[('All Segments', 'all'), 
                     ('Original Segments Only', 'original'), 
                     ('Parallel Segments Only', 'parallel')],
            value='all',
            description='Segments:',
            style={'description_width': 'initial'}
        )
    else:
        segment_filter = None
    
    analyze_selection_button = widgets.Button(
        description='Analyze Selection',
        button_style='info'
    )
    
    analyze_all_button = widgets.Button(
        description='Analyze All',
        button_style='success'
    )
    
    clear_button = widgets.Button(
        description='Clear Selection',
        button_style='warning'
    )
    
    stats_output = widgets.Output()
    plot_output = widgets.Output()
    
    # Toggle comparison mode
    def on_comparison_toggle(change):
        time_selection2.layout.display = 'flex' if change.new else 'none'
    
    comparison_mode.observe(on_comparison_toggle, names='value')

    def analyze_data(gdf_to_analyze):
        """Analyze data based on selected metrics and time periods"""
        selected_metric = metric_selector.value
        selected_dates1 = time_selection1.value
        selected_dates2 = time_selection2.children[0].value if comparison_mode.value else None
        
        # Filter by segment type if applicable
        if segment_filter is not None:
            filter_value = segment_filter.value
            if filter_value == 'original':
                gdf_to_analyze = gdf_to_analyze[gdf_to_analyze['is_original'] == True]
            elif filter_value == 'parallel':
                gdf_to_analyze = gdf_to_analyze[gdf_to_analyze['is_original'] == False]
        
        if not selected_dates1 or (comparison_mode.value and not selected_dates2):
            with stats_output:
                stats_output.clear_output()
                print("Please select time periods for comparison")
            return
                
        with plot_output:
            plot_output.clear_output()
            result1 = process_temporal_data(gdf_to_analyze, selected_metric, selected_dates1)
            result2 = process_temporal_data(gdf_to_analyze, selected_metric, selected_dates2) if comparison_mode.value else None
            
            if result1 is None or (comparison_mode.value and result2 is None):
                print("No data available for selected combination")
                return
            
            if comparison_mode.value:
                # Comparison mode visualizations
                if selected_metric == 'General Comparison':
                    fig = plt.figure(figsize=(20, 12))
                    
                    # Speed distributions
                    plt.subplot(2, 4, 1)
                    plt.bar(range(len(result1['speeds'])), result1['speeds'])
                    plt.title(f'Speed Distribution\n{", ".join(selected_dates1)}')
                    plt.xlabel('Speed (km/h)')
                    plt.ylabel('Count')
                    
                    plt.subplot(2, 4, 2)
                    plt.bar(range(len(result2['speeds'])), result2['speeds'])
                    plt.title(f'Speed Distribution\n{", ".join(selected_dates2)}')
                    plt.xlabel('Speed (km/h)')
                    plt.ylabel('Count')
                    
                    # Other metrics side by side
                    metrics = ['cv', 'ent', 'skew']
                    for i, m in enumerate(metrics):
                        plt.subplot(2, 4, i+3)
                        plt.hist([result1[m], result2[m]], bins=30, label=[', '.join(selected_dates1), ', '.join(selected_dates2)])
                        plt.title(f'{m.capitalize()} Distribution')
                        plt.xlabel('Value')
                        plt.ylabel('Count')
                        plt.legend()
                    
                    plt.tight_layout()
                    plt.show()
                    # Print comparison statistics
                    with stats_output:
                        stats_output.clear_output()
                        print(f"\nAnalyzing {len(gdf_to_analyze)} features")
                        
                        # Create descriptive period labels
                        period1_label = f"{', '.join(selected_dates1)}"
                        period2_label = f"{', '.join(selected_dates2)}" if selected_dates2 else ""
                        
                        # Calculate route count sums for both periods
                        route_count1 = 0
                        for date in selected_dates1:
                            year = date.split('-')[0][-2:]
                            month = date.split('-')[1]
                            route_count_col = f"{year}-{month}_route_count"
                            if route_count_col in gdf_to_analyze.columns:
                                route_count1 += gdf_to_analyze[route_count_col].sum()
                        
                        route_count2 = 0
                        for date in selected_dates2:
                            year = date.split('-')[0][-2:]
                            month = date.split('-')[1]
                            route_count_col = f"{year}-{month}_route_count"
                            if route_count_col in gdf_to_analyze.columns:
                                route_count2 += gdf_to_analyze[route_count_col].sum()
                        
                        # Use pre-calculated speed values if available
                        if 'speed' in result1 and 'speed' in result2:
                            speed1 = np.mean(result1['speed'])
                            speed2 = np.mean(result2['speed'])
                        else:
                            # Fallback to calculating from histograms
                            speed1 = sum(i * c for i, c in enumerate(result1['speeds'])) / sum(result1['speeds']) if sum(result1['speeds']) > 0 else 0
                            speed2 = sum(i * c for i, c in enumerate(result2['speeds'])) / sum(result2['speeds']) if sum(result2['speeds']) > 0 else 0
                        
                        # Basic stats dictionary
                        stats_dict = {
                            'Period': [period1_label, period2_label],
                            'Speed': [speed1, speed2],
                            'CV': [np.mean(result1['cv']), np.mean(result2['cv'])],
                            'Entropy': [np.mean(result1['ent']), np.mean(result2['ent'])],
                            'Skewness': [np.mean(result1['skew']), np.mean(result2['skew'])],
                            'Route Count': [route_count1, route_count2]
                        }
                        
                        # Add quantiles
                        if 'q1' in result1 and 'q1' in result2:
                            stats_dict['Q1'] = [np.mean(result1['q1']), np.mean(result2['q1'])]
                        if 'q2' in result1 and 'q2' in result2:
                            stats_dict['Median'] = [np.mean(result1['q2']), np.mean(result2['q2'])]
                        if 'q3' in result1 and 'q3' in result2:
                            stats_dict['Q3'] = [np.mean(result1['q3']), np.mean(result2['q3'])]
                        if 'iqr' in result1 and 'iqr' in result2:
                            stats_dict['IQR'] = [np.mean(result1['iqr']), np.mean(result2['iqr'])]
                        
                        # Create the DataFrame with basic stats for display
                        df_stats = pd.DataFrame(stats_dict).round(2)
                        
                        # Display human-readable format
                        print("\nComparison Statistics:")
                        print(df_stats.to_string(index=False))
                        
                        # For Excel copy-paste, combine main statistics and speed percentages in one section
                        print("\nFor Excel (copy entire section below):")
                        print("----------------------------------------")
                        
                        # Combined headers for all statistics
                        headers = ['Period', 'Speed', 'CV', 'Entropy', 'Skewness', 'Route Count']
                        if 'Q1' in stats_dict:
                            headers.extend(['Q1', 'Median', 'Q3', 'IQR'])
                        
                        # Add speed percentage headers
                        if all(key in result1 for key in ['p_0_5', 'p_5_15', 'p_15_25', 'p_25p']) and all(key in result2 for key in ['p_0_5', 'p_5_15', 'p_15_25', 'p_25p']):
                            headers.extend(['0-5 km/h', '5-15 km/h', '15-25 km/h', '25+ km/h'])
                        
                        print('\t'.join(headers))
                        
                        # First data row with all metrics
                        row1 = [period1_label, f"{speed1:.2f}", f"{np.mean(result1['cv']):.2f}", 
                                f"{np.mean(result1['ent']):.2f}", f"{np.mean(result1['skew']):.2f}",
                                f"{route_count1}"]
                        
                        # Add quantiles if available
                        if 'Q1' in stats_dict:
                            row1.extend([f"{np.mean(result1['q1']):.2f}", f"{np.mean(result1['q2']):.2f}", 
                                        f"{np.mean(result1['q3']):.2f}", f"{np.mean(result1['iqr']):.2f}"])
                        
                        # Add speed percentages if available
                        if all(key in result1 for key in ['p_0_5', 'p_5_15', 'p_15_25', 'p_25p']):
                            row1.extend([
                                f"{np.mean(result1['p_0_5']):.2f}", 
                                f"{np.mean(result1['p_5_15']):.2f}", 
                                f"{np.mean(result1['p_15_25']):.2f}", 
                                f"{np.mean(result1['p_25p']):.2f}"
                            ])
                        
                        print('\t'.join(row1))
                        
                        # Second data row with all metrics
                        row2 = [period2_label, f"{speed2:.2f}", f"{np.mean(result2['cv']):.2f}", 
                                f"{np.mean(result2['ent']):.2f}", f"{np.mean(result2['skew']):.2f}",
                                f"{route_count2}"]
                        
                        # Add quantiles if available
                        if 'Q1' in stats_dict:
                            row2.extend([f"{np.mean(result2['q1']):.2f}", f"{np.mean(result2['q2']):.2f}", 
                                        f"{np.mean(result2['q3']):.2f}", f"{np.mean(result2['iqr']):.2f}"])
                        
                        # Add speed percentages if available
                        if all(key in result2 for key in ['p_0_5', 'p_5_15', 'p_15_25', 'p_25p']):
                            row2.extend([
                                f"{np.mean(result2['p_0_5']):.2f}", 
                                f"{np.mean(result2['p_5_15']):.2f}", 
                                f"{np.mean(result2['p_15_25']):.2f}", 
                                f"{np.mean(result2['p_25p']):.2f}"
                            ])
                        
                        print('\t'.join(row2))
                        print("----------------------------------------")
                        
                elif selected_metric == 'speeds':
                    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
                    
                    ax1.bar(range(len(result1)), result1)
                    ax1.set_title(f'Speed Distribution\n{", ".join(selected_dates1)}')
                    ax1.set_xlabel('Speed (km/h)')
                    ax1.set_ylabel('Count')
                    ax1.grid(True, alpha=0.3)
                    
                    ax2.bar(range(len(result2)), result2)
                    ax2.set_title(f'Speed Distribution\n{", ".join(selected_dates2)}')
                    ax2.set_xlabel('Speed (km/h)')
                    ax2.set_ylabel('Count')
                    ax2.grid(True, alpha=0.3)
                    
                    plt.tight_layout()
                    plt.show()
                    
                    with stats_output:
                        stats_output.clear_output()
                        print(f"\nAnalyzing {len(gdf_to_analyze)} features")
                        
                        # Calculate route counts
                        route_count1 = 0
                        for date in selected_dates1:
                            year = date.split('-')[0][-2:]
                            month = date.split('-')[1]
                            route_count_col = f"{year}-{month}_route_count"
                            if route_count_col in gdf_to_analyze.columns:
                                route_count1 += gdf_to_analyze[route_count_col].sum()
                        
                        route_count2 = 0
                        for date in selected_dates2:
                            year = date.split('-')[0][-2:]
                            month = date.split('-')[1]
                            route_count_col = f"{year}-{month}_route_count"
                            if route_count_col in gdf_to_analyze.columns:
                                route_count2 += gdf_to_analyze[route_count_col].sum()
                        
                        print(f"\n{', '.join(selected_dates1)}:")
                        print(f"Total measurements: {sum(result1):,}")
                        print(f"Route count: {route_count1}")
                        mean_speed1 = sum(i * count for i, count in enumerate(result1)) / sum(result1) if sum(result1) > 0 else 0
                        print(f"Mean speed: {mean_speed1:.2f} km/h")
                        
                        print(f"\n{', '.join(selected_dates2)}:")
                        print(f"Total measurements: {sum(result2):,}")
                        print(f"Route count: {route_count2}")
                        mean_speed2 = sum(i * count for i, count in enumerate(result2)) / sum(result2) if sum(result2) > 0 else 0
                        print(f"Mean speed: {mean_speed2:.2f} km/h")
                        
                        # Excel copy-paste format
                        print("\nFor Excel (copy entire section below):")
                        print("----------------------------------------")
                        print('\t'.join(['Period', 'Measurements', 'Route Count', 'Mean Speed']))
                        print(f"{', '.join(selected_dates1)}\t{sum(result1)}\t{route_count1}\t{mean_speed1:.2f}")
                        print(f"{', '.join(selected_dates2)}\t{sum(result2)}\t{route_count2}\t{mean_speed2:.2f}")
                        print("----------------------------------------")
                
                else:
                    # Other metrics comparison
                    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
                    
                    ax1.hist(result1, bins=30)
                    ax1.set_title(f'{selected_metric.capitalize()} Distribution\n{", ".join(selected_dates1)}')
                    ax1.set_xlabel('Value')
                    ax1.set_ylabel('Count')
                    ax1.grid(True, alpha=0.3)
                    
                    ax2.hist(result2, bins=30)
                    ax2.set_title(f'{selected_metric.capitalize()} Distribution\n{", ".join(selected_dates2)}')
                    ax2.set_xlabel('Value')
                    ax2.set_ylabel('Count')
                    ax2.grid(True, alpha=0.3)
                    
                    plt.tight_layout()
                    plt.show()
                    
                    with stats_output:
                        stats_output.clear_output()
                        print(f"\nAnalyzing {len(gdf_to_analyze)} features")
                        
                        # Calculate route counts
                        route_count1 = 0
                        for date in selected_dates1:
                            year = date.split('-')[0][-2:]
                            month = date.split('-')[1]
                            route_count_col = f"{year}-{month}_route_count"
                            if route_count_col in gdf_to_analyze.columns:
                                route_count1 += gdf_to_analyze[route_count_col].sum()
                                
                        route_count2 = 0
                        for date in selected_dates2:
                            year = date.split('-')[0][-2:]
                            month = date.split('-')[1]
                            route_count_col = f"{year}-{month}_route_count"
                            if route_count_col in gdf_to_analyze.columns:
                                route_count2 += gdf_to_analyze[route_count_col].sum()
                        
                        print(f"\n{', '.join(selected_dates1)} {selected_metric}: {np.mean(result1):.2f}")
                        print(f"Route count: {route_count1}")
                        print(f"{', '.join(selected_dates2)} {selected_metric}: {np.mean(result2):.2f}")
                        print(f"Route count: {route_count2}")
                        
                        # Excel copy-paste format
                        print("\nFor Excel (copy entire section below):")
                        print("----------------------------------------")
                        print('\t'.join(['Period', selected_metric, 'Route Count']))
                        print(f"{', '.join(selected_dates1)}\t{np.mean(result1):.2f}\t{route_count1}")
                        print(f"{', '.join(selected_dates2)}\t{np.mean(result2):.2f}\t{route_count2}")
                        print("----------------------------------------")

            else:
                # Single period visualizations
                if selected_metric == 'General Comparison':
                    plt.figure(figsize=(15, 10))
                    
                    plt.subplot(2, 2, 1)
                    plt.bar(range(len(result1['speeds'])), result1['speeds'])
                    plt.title(f'Speed Distribution\n{", ".join(selected_dates1)}')
                    plt.xlabel('Speed (km/h)')
                    plt.ylabel('Count')
                    
                    metrics = ['cv', 'ent', 'skew']
                    for i, m in enumerate(metrics, 2):
                        plt.subplot(2, 2, i)
                        plt.hist(result1[m], bins=30)
                        plt.title(f'{m.capitalize()} Distribution')
                        plt.xlabel('Value')
                        plt.ylabel('Count')
                    
                    plt.tight_layout()
                    plt.show()
                    
                    with stats_output:
                        stats_output.clear_output()
                        print(f"\nAnalyzing {len(gdf_to_analyze)} features")
                        
                        # Calculate route count sum
                        route_count = 0
                        for date in selected_dates1:
                            year = date.split('-')[0][-2:]
                            month = date.split('-')[1]
                            route_count_col = f"{year}-{month}_route_count"
                            if route_count_col in gdf_to_analyze.columns:
                                route_count += gdf_to_analyze[route_count_col].sum()
                        
                        # Use pre-calculated speed values if available
                        if 'speed' in result1:
                            speed = np.mean(result1['speed'])
                        else:
                            # Fallback to calculating from histograms
                            speed = sum(i * c for i, c in enumerate(result1['speeds'])) / sum(result1['speeds']) if sum(result1['speeds']) > 0 else 0
                        
                        # Basic stats dictionary (using traditional format for human readability)
                        stats_dict = {
                            'Speed': speed,
                            'CV': np.mean(result1['cv']),
                            'Entropy': np.mean(result1['ent']),
                            'Skewness': np.mean(result1['skew']),
                            'Route Count': route_count
                        }
                        
                        # Add quantiles if available
                        if 'q1' in result1:
                            stats_dict['Q1'] = np.mean(result1['q1'])
                        if 'q2' in result1:
                            stats_dict['Median'] = np.mean(result1['q2'])
                        if 'q3' in result1:
                            stats_dict['Q3'] = np.mean(result1['q3'])
                        if 'iqr' in result1:
                            stats_dict['IQR'] = np.mean(result1['iqr'])
                        
                        # Display human-readable format
                        print("\nMetric Statistics:")
                        df_stats = pd.DataFrame([stats_dict]).round(2)
                        print(df_stats.to_string())
                        
                        # For Excel copy-paste, combine all statistics in one section
                        print("\nFor Excel (copy entire section below):")
                        print("----------------------------------------")
                        
                        # Create a comprehensive table with metrics and speed percentages in one section
                        # Start with the main metrics
                        metrics = ['Speed', 'CV', 'Entropy', 'Skewness', 'Route Count']
                        
                        # Add quantiles if available
                        if 'Q1' in stats_dict:
                            metrics.extend(['Q1', 'Median', 'Q3', 'IQR'])
                        
                        # Add speed percentage metrics if available
                        if all(key in result1 for key in ['p_0_5', 'p_5_15', 'p_15_25', 'p_25p']):
                            metrics.extend(['0-5 km/h', '5-15 km/h', '15-25 km/h', '25+ km/h'])
                        
                        # Create the header
                        print('\t'.join(['Metric', 'Value']))
                        
                        # Print values for each metric
                        for metric in metrics:
                            if metric == 'Speed':
                                value = f"{stats_dict['Speed']:.2f}"
                            elif metric == 'CV':
                                value = f"{stats_dict['CV']:.2f}"
                            elif metric == 'Entropy':
                                value = f"{stats_dict['Entropy']:.2f}"
                            elif metric == 'Skewness':
                                value = f"{stats_dict['Skewness']:.2f}"
                            elif metric == 'Route Count':
                                value = f"{stats_dict['Route Count']}"
                            # Quantiles
                            elif metric in ['Q1', 'Median', 'Q3', 'IQR'] and metric in stats_dict:
                                value = f"{stats_dict[metric]:.2f}"
                            # Speed ranges    
                            elif metric == '0-5 km/h' and 'p_0_5' in result1:
                                value = f"{np.mean(result1['p_0_5']):.2f}"
                            elif metric == '5-15 km/h' and 'p_5_15' in result1:
                                value = f"{np.mean(result1['p_5_15']):.2f}" 
                            elif metric == '15-25 km/h' and 'p_15_25' in result1:
                                value = f"{np.mean(result1['p_15_25']):.2f}"
                            elif metric == '25+ km/h' and 'p_25p' in result1:
                                value = f"{np.mean(result1['p_25p']):.2f}"
                            else:
                                value = "N/A"
                                
                            print(f"{metric}\t{value}")
                            
                        print("----------------------------------------")

                elif selected_metric == 'speeds':
                    plt.figure(figsize=(12, 6))
                    plt.bar(range(len(result1)), result1)
                    plt.title(f'Speed Distribution\n({", ".join(selected_dates1)})')
                    plt.xlabel('Speed (km/h)')
                    plt.ylabel('Count')
                    plt.grid(True, alpha=0.3)
                    plt.show()
                    
                    with stats_output:
                        stats_output.clear_output()
                        print(f"\nAnalyzing {len(gdf_to_analyze)} features")
                        
                        # Calculate route count
                        route_count = 0
                        for date in selected_dates1:
                            year = date.split('-')[0][-2:]
                            month = date.split('-')[1]
                            route_count_col = f"{year}-{month}_route_count"
                            if route_count_col in gdf_to_analyze.columns:
                                route_count += gdf_to_analyze[route_count_col].sum()
                                
                        print(f"Total measurements: {sum(result1):,}")
                        print(f"Route count: {route_count}")
                        mean_speed = sum(i * count for i, count in enumerate(result1)) / sum(result1) if sum(result1) > 0 else 0
                        print(f"Mean speed: {mean_speed:.2f} km/h")
                        
                        # Excel copy-paste format
                        print("\nFor Excel (copy entire section below):")
                        print("----------------------------------------")
                        print('\t'.join(['Measurements', 'Route Count', 'Mean Speed']))
                        print(f"{sum(result1)}\t{route_count}\t{mean_speed:.2f}")
                        print("----------------------------------------")
                
                else:
                    plt.figure(figsize=(12, 6))
                    plt.hist(result1, bins=30)
                    plt.title(f'{selected_metric.capitalize()} Distribution\n({", ".join(selected_dates1)})')
                    plt.xlabel('Value')
                    plt.ylabel('Count')
                    plt.grid(True, alpha=0.3)
                    plt.show()
                    
                    with stats_output:
                        stats_output.clear_output()
                        print(f"\nAnalyzing {len(gdf_to_analyze)} features")
                        
                        # Calculate route count
                        route_count = 0
                        for date in selected_dates1:
                            year = date.split('-')[0][-2:]
                            month = date.split('-')[1]
                            route_count_col = f"{year}-{month}_route_count"
                            if route_count_col in gdf_to_analyze.columns:
                                route_count += gdf_to_analyze[route_count_col].sum()
                        
                        print(f"\n{selected_metric}: {np.mean(result1):.2f}")
                        print(f"Route count: {route_count}")
                        
                        # Excel copy-paste format
                        print("\nFor Excel (copy entire section below):")
                        print("----------------------------------------")
                        print('\t'.join(['Metric', 'Value', 'Route Count']))
                        print(f"{selected_metric}\t{np.mean(result1):.2f}\t{route_count}")
                        print("----------------------------------------")

    def on_analyze_selection_click(b):
        try:
            drawn_features = draw_control.data
            if not drawn_features:
                with stats_output:
                    stats_output.clear_output()
                    print("Please draw at least one shape")
                return
            
            selection_shapes = [shape(feature['geometry']) for feature in drawn_features]
            combined_shape = selection_shapes[0]
            for s in selection_shapes[1:]:
                combined_shape = combined_shape.union(s)
            
            gdf_wgs84 = ORIGINAL_GDF.to_crs('EPSG:4326')
            selected_gdf = gdf_wgs84[gdf_wgs84.intersects(combined_shape)]
            
            if len(selected_gdf) == 0:
                with stats_output:
                    stats_output.clear_output()
                    print("No features found in selection")
                return
            
            analyze_data(selected_gdf)
            
        except Exception as e:
            with stats_output:
                stats_output.clear_output()
                print(f"Error analyzing selection: {str(e)}")

    def on_analyze_all_click(b):
        try:
            analyze_data(ORIGINAL_GDF.to_crs('EPSG:4326'))
        except Exception as e:
            with stats_output:
                stats_output.clear_output()
                print(f"Error analyzing all features: {str(e)}")
    
    def on_clear_click(b):
        draw_control.clear()
        stats_output.clear_output()
        plot_output.clear_output()
        
    analyze_selection_button.on_click(on_analyze_selection_click)
    analyze_all_button.on_click(on_analyze_all_click)
    clear_button.on_click(on_clear_click)
    
    # Arrange widgets
    control_widgets = [
        widgets.HTML(value="<b>Analysis Controls:</b>"),
        comparison_mode,
        metric_selector,
        time_selection1,
        time_selection2
    ]
    
    # Add segment filter if available
    if segment_filter is not None:
        control_widgets.insert(3, segment_filter)
    
    control_widgets.append(widgets.HBox([analyze_selection_button, analyze_all_button, clear_button]))
    control_widgets.append(stats_output)
    control_widgets.append(plot_output)
    
    controls = widgets.VBox(control_widgets)
    
    return controls

def initialize_analysis(gdf, original_gdf=None):
    """Initialize the analysis environment with optional original geometries
    
    Parameters:
    -----------
    gdf : GeoDataFrame
        The segments to analyze (gdf3)
    original_gdf : GeoDataFrame, optional
        The original reference geometry
    """
    # Create map and controls
    # m, draw_control = create_interactive_map(gdf, original_gdf)
    m, draw_control = create_interactive_map(gdf)
    controls = create_analysis_widgets(gdf, m, draw_control)
    
    # Display the map and controls
    display(m)
    display(controls)
    
    # Don't return anything to avoid "None" output
    return None

# Initialize the analysis tools with your existing gdf3 data
# Here's how you would use it with original gdf overlay:
# initialize_analysis(gdf3, original_gdf=gdf)

# Or without original reference geometry:
initialize_analysis(gdf3)

Map(center=[np.float64(52.505145999999996), np.float64(13.408959)], controls=(ZoomControl(options=['position',…

In [6]:
# Step 3 v4.3 final: [FULL] Analyze values by month interactively with length weighting + processing statistical values on the fly

import geopandas as gpd
import pandas as pd
import numpy as np
import ipyleaflet
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import json
import warnings
from shapely.geometry import Point, shape
from scipy.stats import entropy
from typing import Optional, Tuple
from IPython.display import display
import ipywidgets as widgets

# Speed Statistics Calculator Class
class SpeedStatistics:
    """Class to handle speed histogram statistics calculations"""
    
    def __init__(self, n_bins: int = 32):
        self.n_bins = n_bins
        self.bin_edges = np.arange(0, n_bins + 1)
        self.midpoints = (self.bin_edges[:-1] + self.bin_edges[1:]) / 2
    
    def parse_histogram(self, binned_speeds) -> Optional[np.ndarray]:
        """Safely parse histogram string to numpy array"""
        if binned_speeds is None:
            return None
        
        if isinstance(binned_speeds, (list, np.ndarray)):
            # If already a list or array, convert to numpy array
            return np.array(binned_speeds)
            
        if pd.isna(binned_speeds):
            return None
            
        try:
            if isinstance(binned_speeds, str):
                hist = eval(binned_speeds)
            else:
                hist = binned_speeds
                
            hist_array = np.array(hist)
            if len(hist_array) != self.n_bins:
                warnings.warn(f"Expected {self.n_bins} bins, got {len(hist_array)}")
                return None
            return hist_array
            
        except Exception as e:
            warnings.warn(f"Error parsing histogram: {e}")
            return None
    
    def calculate_stats(self, binned_speeds, length_m: float = 1.0) -> Tuple:
        """Calculate comprehensive statistics for a speed histogram with length weighting"""
        # Parse histogram
        hist_array = self.parse_histogram(binned_speeds)
        if hist_array is None:
            return (np.nan,) * 15
        
        # Apply length weighting
        weighted_hist = hist_array * length_m
        total_count = weighted_hist.sum()
        if total_count == 0:
            return (np.nan,) * 15
            
        # Calculate probabilities
        probs = weighted_hist / total_count
        
        # Basic statistics
        weighted_mean = np.sum(self.midpoints * probs)
        weighted_var = np.sum(((self.midpoints - weighted_mean) ** 2) * probs)
        weighted_std = np.sqrt(weighted_var)
        
        # Higher moments
        if weighted_std > 0:
            standardized_moments = (self.midpoints - weighted_mean) / weighted_std
            skewness = np.sum((standardized_moments ** 3) * probs)
            kurtosis = np.sum((standardized_moments ** 4) * probs) - 3
        else:
            skewness = np.nan
            kurtosis = np.nan
        
        # Calculate CV
        cv = (weighted_std / weighted_mean * 100) if weighted_mean > 0 else np.nan
        
        # Entropy (only for non-zero probabilities)
        valid_probs = probs[probs > 0]
        data_entropy = entropy(valid_probs, base=2) if len(valid_probs) > 0 else np.nan
        
        # Quantiles
        cum_probs = np.cumsum(weighted_hist) / total_count
        q1 = np.interp(0.25, cum_probs, self.midpoints)
        q2 = np.interp(0.50, cum_probs, self.midpoints)
        q3 = np.interp(0.75, cum_probs, self.midpoints)
        q4 = np.interp(1.00, cum_probs, self.midpoints)
        iqr = q3 - q1
        
        # Speed range percentages
        pct_0_5 = np.interp(5, self.midpoints, cum_probs, left=0) * 100
        pct_5_15 = (np.interp(15, self.midpoints, cum_probs) - 
                   np.interp(5, self.midpoints, cum_probs)) * 100
        pct_15_25 = (np.interp(25, self.midpoints, cum_probs) - 
                    np.interp(15, self.midpoints, cum_probs)) * 100
        pct_25_plus = (1 - np.interp(25, self.midpoints, cum_probs, right=1)) * 100
        
        return tuple(round(x, 2) if not pd.isna(x) else x for x in [
            weighted_mean, weighted_std, cv, skewness, kurtosis,
            data_entropy, iqr, q1, q2, q3, q4,
            pct_0_5, pct_5_15, pct_15_25, pct_25_plus
        ])

# Helper function to format date ranges
def format_date_range(dates):
    """Format a list of dates into a range string"""
    if not dates:
        return ""
    
    # Convert to datetime and sort
    date_objs = sorted([pd.to_datetime(date) for date in dates])
    
    # Get the first and last date
    first_date = date_objs[0]
    last_date = date_objs[-1]
    
    # Format as "YY-MM to YY-MM"
    return f"{first_date.strftime('%y-%m')} to {last_date.strftime('%y-%m')}"

# Get available metrics and times for analysis
def get_available_metrics_and_times(gdf):
    """Get available metrics and time periods from the data"""
    # Find all columns that match the pattern YY-MM_metric
    time_cols = sorted([col for col in gdf.columns 
                        if len(col.split('-')) > 1 and 
                        len(col.split('-')[0]) == 2 and 
                        len(col.split('-')[1].split('_')[0]) == 2])
    
    metrics = set()
    time_periods = set()
    
    for col in time_cols:
        parts = col.split('_', 1)
        if len(parts) > 1:
            yy_mm = parts[0]
            metric = parts[1]
            
            yy, mm = yy_mm.split('-')
            time_periods.add(f"20{yy}-{mm}")
            metrics.add(metric)
    
    # Define core metrics (even if they don't exist in the data)
    # We'll calculate these on-the-fly from speeds histograms if needed
    core_metrics = {'speeds', 'speed', 'route_count', 'cv', 'ent', 'skew', 'kurt', 'std', 'iqr', 
                    'q1', 'q2', 'q3', 'q4', 'p_0_5', 'p_5_15', 'p_15_25', 'p_25p'}
    
    # Only include metrics that have corresponding speed or speeds histograms
    available_metrics = {'route_count'}
    for period in time_periods:
        yy = period.split('-')[0][-2:]
        mm = period.split('-')[1]
        if f"{yy}-{mm}_speed" in gdf.columns:
            available_metrics.add('speed')
        if f"{yy}-{mm}_speeds" in gdf.columns:
            available_metrics.add('speeds')
        if f"{yy}-{mm}_cv" in gdf.columns:
            available_metrics.add('cv')
        if f"{yy}-{mm}_ent" in gdf.columns:
            available_metrics.add('ent')
        if f"{yy}-{mm}_skew" in gdf.columns:
            available_metrics.add('skew')
    
    # Add derived metrics that we can calculate if not already in the data
    derived_metrics = {'speed', 'cv', 'ent', 'skew', 'kurt', 'std', 'iqr', 
                      'q1', 'q2', 'q3', 'q4', 'p_0_5', 'p_5_15', 'p_15_25', 'p_25p'}
    
    # Merge available and derived metrics
    filtered_metrics = available_metrics.union(derived_metrics)
    
    return sorted(list(filtered_metrics)), sorted(list(time_periods))

# Process temporal data for analysis
def process_temporal_data(gdf, metric, selected_dates):
    """Process temporal data for given metric and dates with length weighting"""
    if not metric or not selected_dates:
        return None
        
    # Initialize the speed statistics calculator
    calculator = SpeedStatistics()
    
    if metric == 'General Comparison':
        # Expanded metrics list to include quantiles and speed ranges
        metrics = ['speeds', 'cv', 'ent', 'skew', 'q1', 'q2', 'q3', 'iqr', 'p_0_5', 'p_5_15', 'p_15_25', 'p_25p']
        results = {}
        
        # First get all the pre-calculated speed values
        speed_values = []
        for date in selected_dates:
            year = date.split('-')[0][-2:]
            month = date.split('-')[1]
            speed_col = f"{year}-{month}_speed"
            if speed_col in gdf.columns:
                values = gdf[speed_col].dropna().tolist()
                speed_values.extend(values)
        
        if speed_values:
            results['speed'] = speed_values
            
        # For speeds histograms, sum them with length weighting
        histograms = []
        lengths = []
        for date in selected_dates:
            year = date.split('-')[0][-2:]
            month = date.split('-')[1]
            speeds_col = f"{year}-{month}_speeds"
            if speeds_col in gdf.columns:
                for idx, hist_str in enumerate(gdf[speeds_col]):
                    if pd.notna(hist_str):
                        hist = eval(hist_str) if isinstance(hist_str, str) else hist_str
                        histograms.append(hist)
                        lengths.append(gdf.iloc[idx]['length_m'] if 'length_m' in gdf.columns else 1.0)
        
        if histograms:
            # First ensure all histograms have the same length (should be 32 based on SpeedStatistics class)
            expected_length = 32
            uniform_histograms = []
            
            for hist, length in zip(histograms, lengths):
                # Check if the histogram has the expected length
                if len(hist) == expected_length:
                    uniform_histograms.append(np.array(hist) * length)
                else:
                    # Log warning and skip or pad the histogram
                    warnings.warn(f"Histogram with unexpected length {len(hist)} found. Expected {expected_length}.")
                    # Option 1: Skip this histogram
                    # continue
                    
                    # Option 2: Pad or truncate the histogram to expected length
                    if len(hist) < expected_length:
                        # Pad with zeros
                        padded_hist = np.pad(hist, (0, expected_length - len(hist)), 'constant')
                        uniform_histograms.append(padded_hist * length)
                    else:
                        # Truncate
                        uniform_histograms.append(np.array(hist[:expected_length]) * length)
    
            # Only sum if we have valid histograms
            if uniform_histograms:
                # Convert to numpy array first to ensure proper shape for summation
                uniform_array = np.array(uniform_histograms)
                results['speeds'] = np.sum(uniform_array, axis=0)
            else:
                # If no valid histograms, just create an empty one
                results['speeds'] = np.zeros(expected_length)
            
        # Metric index in stats tuple
        metric_index = {
            'cv': 2,       # coefficient of variation
            'skew': 3,     # skewness
            'ent': 5,      # entropy
            'iqr': 6,      # interquartile range
            'q1': 7,       # first quartile
            'q2': 8,       # median
            'q3': 9,       # third quartile
            'p_0_5': 11,   # % speeds 0-5 km/h
            'p_5_15': 12,  # % speeds 5-15 km/h
            'p_15_25': 13, # % speeds 15-25 km/h
            'p_25p': 14    # % speeds 25+ km/h
        }
        
        # Try using pre-calculated metrics first, then calculate from histograms if needed
        for m in metrics:
            if m == 'speeds':
                continue  # Already handled above
                
            values = []
            for date in selected_dates:
                year = date.split('-')[0][-2:]
                month = date.split('-')[1]
                metric_col = f"{year}-{month}_{m}"
                if metric_col in gdf.columns:
                    metric_values = gdf[metric_col].dropna().tolist()
                    values.extend(metric_values)
            
            if values:
                results[m] = values
            elif histograms and m in metric_index:
                # Calculate from histograms if we don't have pre-calculated values
                calculated_values = []
                for hist, length in zip(histograms, lengths):
                    stats = calculator.calculate_stats(hist, length)
                    if not pd.isna(stats[metric_index[m]]):
                        calculated_values.append(stats[metric_index[m]])
                
                if calculated_values:
                    results[m] = calculated_values
        
        return results if results else None
    
    elif metric == 'speed':
        # Try to use pre-calculated speed values first
        values = []
        for date in selected_dates:
            year = date.split('-')[0][-2:]
            month = date.split('-')[1]
            col = f"{year}-{month}_speed"
            if col in gdf.columns:
                values.extend(gdf[col].dropna().tolist())
        
        if values:
            return values
        
        # If no pre-calculated values, calculate from histograms
        values = []
        for date in selected_dates:
            year = date.split('-')[0][-2:]
            month = date.split('-')[1]
            speeds_col = f"{year}-{month}_speeds"
            
            if speeds_col in gdf.columns:
                for hist_str in gdf[speeds_col]:
                    if pd.notna(hist_str):
                        stats = calculator.calculate_stats(hist_str, gdf.loc[gdf[speeds_col] == hist_str, 'length_m'].values[0] if 'length_m' in gdf.columns else 1.0)
                        if not pd.isna(stats[0]):  # Mean speed is the first value
                            values.append(stats[0])
        
        return values if values else None
    
    elif metric == 'speeds':
        # Sum all histograms for the speeds metric with length weighting
        histograms = []
        lengths = []
        for date in selected_dates:
            year = date.split('-')[0][-2:]
            month = date.split('-')[1]
            col = f"{year}-{month}_speeds"
            if col in gdf.columns:
                for idx, hist_str in enumerate(gdf[col]):
                    if pd.notna(hist_str):
                        hist = eval(hist_str) if isinstance(hist_str, str) else hist_str
                        histograms.append(hist)
                        lengths.append(gdf.iloc[idx]['length_m'] if 'length_m' in gdf.columns else 1.0)
        return np.sum([hist * length for hist, length in zip(histograms, lengths)], axis=0) if histograms else None
    
    elif metric in ['cv', 'ent', 'skew', 'kurt', 'std', 'iqr', 
                    'q1', 'q2', 'q3', 'q4', 'p_0_5', 'p_5_15', 'p_15_25', 'p_25p']:
        # First try to get pre-calculated values
        values = []
        for date in selected_dates:
            year = date.split('-')[0][-2:]
            month = date.split('-')[1]
            col = f"{year}-{month}_{metric}"
            if col in gdf.columns:
                values.extend(gdf[col].dropna().tolist())
        
        if values:
            return values
        
        # If no pre-calculated values, calculate from histograms
        metric_index = {
            'speed': 0,    # mean
            'std': 1,      # standard deviation
            'cv': 2,       # coefficient of variation  
            'skew': 3,     # skewness
            'kurt': 4,     # kurtosis
            'ent': 5,      # entropy
            'iqr': 6,      # interquartile range
            'q1': 7,       # first quartile
            'q2': 8,       # median
            'q3': 9,       # third quartile
            'q4': 10,      # maximum
            'p_0_5': 11,   # % speeds 0-5 km/h
            'p_5_15': 12,  # % speeds 5-15 km/h
            'p_15_25': 13, # % speeds 15-25 km/h
            'p_25p': 14    # % speeds 25+ km/h
        }
        
        index = metric_index.get(metric, 0)
        values = []
        
        for date in selected_dates:
            year = date.split('-')[0][-2:]
            month = date.split('-')[1]
            speeds_col = f"{year}-{month}_speeds"
            
            if speeds_col in gdf.columns:
                for hist_str in gdf[speeds_col]:
                    if pd.notna(hist_str):
                        stats = calculator.calculate_stats(hist_str, gdf.loc[gdf[speeds_col] == hist_str, 'length_m'].values[0] if 'length_m' in gdf.columns else 1.0)
                        if not pd.isna(stats[index]):
                            values.append(stats[index])
        
        return values if values else None
    
    else:
        # For other metrics like route_count that may already exist
        values = []
        for date in selected_dates:
            year = date.split('-')[0][-2:]
            month = date.split('-')[1]
            col = f"{year}-{month}_{metric}"
            if col in gdf.columns:
                values.extend(gdf[col].dropna().tolist())
        return values if values else None

# Create interactive map
def create_interactive_map(gdf, original_gdf=None):
    """Create interactive map with draw controls"""
    gdf_wgs84 = gdf.to_crs('EPSG:4326')
    center_lat = gdf_wgs84.total_bounds[[1,3]].mean()
    center_lon = gdf_wgs84.total_bounds[[0,2]].mean()
    
    m = ipyleaflet.Map(
        center=(center_lat, center_lon),
        zoom=14,
        scroll_wheel_zoom=True
    )
    
    draw_control = ipyleaflet.DrawControl(
        polygon={"shapeOptions": {"color": "#ff0000"}},
        rectangle={"shapeOptions": {"color": "#ff0000"}},
        circle={"shapeOptions": {"color": "#ff0000"}},
        circlemarker={},
        polyline={},
        marker={}
    )
    m.add_control(draw_control)
    
    # Style based on segment type if 'is_original' column exists
    if 'is_original' in gdf_wgs84.columns:
        # Speed field for coloring
        speed_field = '2304-2412_speed'
        if speed_field not in gdf_wgs84.columns:
            speed_fields = [col for col in gdf_wgs84.columns if col.endswith('_speed')]
            if speed_fields:
                speed_field = speed_fields[0]
            else:
                speed_field = None
        
        # Setup color normalization for speed values if available
        if speed_field:
            valid_speeds = gdf_wgs84[gdf_wgs84['is_original'] == False][speed_field].dropna()
            if len(valid_speeds) > 0:
                norm = plt.Normalize(valid_speeds.min(), valid_speeds.max())
            else:
                norm = plt.Normalize(0, 30)
            cmap = plt.cm.viridis
        
        def style_callback(feature):
            # Check if this is an original segment
            is_original = feature['properties'].get('is_original', False)
            
            if is_original:
                return {
                    'color': '#FF0000',  # Red for original segments
                    'weight': 5,
                    'opacity': 1.0,
                    'dashArray': '5,5'  # Dashed line for original segments
                }
            else:
                # For parallel segments, use speed-based coloring if available
                if speed_field and speed_field in feature['properties']:
                    speed = feature['properties'][speed_field]
                    if pd.notna(speed):
                        color = mcolors.rgb2hex(cmap(norm(speed)))
                    else:
                        color = '#808080'  # Gray for unknown speed
                else:
                    color = '#0000FF'  # Blue for parallel segments with no speed data
                    
                return {
                    'color': color,
                    'weight': 3,
                    'opacity': 0.7
                }
    else:
        # Use the standard speed-based styling
        # Use the new field name as specified
        speed_field = '2304-2412_speed'
        
        # If speed field doesn't exist, calculate it from the speed histogram
        if speed_field not in gdf_wgs84.columns and '2304-2412_speeds' in gdf_wgs84.columns:
            calculator = SpeedStatistics()
            gdf_wgs84[speed_field] = gdf_wgs84['2304-2412_speeds'].apply(
                lambda x: calculator.calculate_stats(x)[0] if pd.notna(x) else None
            )
        
        # Fallback options if still not available
        if speed_field not in gdf_wgs84.columns:
            # Try to find any existing speed field
            speed_fields = [col for col in gdf_wgs84.columns if col.endswith('_speed')]
            if speed_fields:
                speed_field = speed_fields[0]
            else:
                # Or calculate from any speeds histogram field
                speeds_fields = [col for col in gdf_wgs84.columns if col.endswith('_speeds')]
                if speeds_fields:
                    speed_field = speeds_fields[0].replace('_speeds', '_speed')
                    calculator = SpeedStatistics()
                    gdf_wgs84[speed_field] = gdf_wgs84[speeds_fields[0]].apply(
                        lambda x: calculator.calculate_stats(x)[0] if pd.notna(x) else None
                    )
                else:
                    # If nothing else, use a default column
                    speed_field = gdf_wgs84.columns[0]
        
        # Filter out NaN values for normalization
        valid_speeds = gdf_wgs84[speed_field].dropna()
        if len(valid_speeds) > 0:
            norm = plt.Normalize(valid_speeds.min(), valid_speeds.max())
        else:
            # Fallback if no valid speed values
            norm = plt.Normalize(0, 30)  # Reasonable default for km/h
        
        cmap = plt.cm.viridis

        def style_callback(feature):
            speed = feature['properties'][speed_field] if speed_field in feature['properties'] else None
            color = '#808080' if pd.isna(speed) else mcolors.rgb2hex(cmap(norm(speed)))
            return {'color': color, 'weight': 3, 'opacity': 0.7}

    geo_json = ipyleaflet.GeoJSON(
        data=json.loads(gdf_wgs84.to_json()),
        style={'weight': 3, 'opacity': 0.7},
        hover_style={'color': 'red', 'fillOpacity': 0.9},
        style_callback=style_callback
    )
    m.add_layer(geo_json)
    
    # Add the original geometries if provided and not already included
    if original_gdf is not None and 'is_original' not in gdf_wgs84.columns:
        # Convert to same CRS
        original_wgs84 = original_gdf.to_crs('EPSG:4326')
        
        # Add as a highlighted layer
        original_json = ipyleaflet.GeoJSON(
            data=json.loads(original_wgs84.to_json()),
            style={
                'color': '#FF0000',  # Red color
                'weight': 5,         # Thicker line
                'opacity': 1.0,      # Fully opaque
                'dashArray': '5,5'   # Dashed line pattern
            },
            hover_style={
                'color': '#FF00FF',  # Magenta on hover
                'weight': 7
            },
            name='Original Reference'
        )
        m.add_layer(original_json)
    
    return m, draw_control

# Create analysis widgets
def create_analysis_widgets(gdf, m, draw_control):
    """Create analysis widgets with comparison mode"""
    global ORIGINAL_GDF
    ORIGINAL_GDF = gdf.copy()
    
    metrics, time_periods = get_available_metrics_and_times(gdf)
    all_metrics = ['General Comparison'] + metrics
    
    # Create widgets
    comparison_mode = widgets.Checkbox(
        value=False,
        description='Comparison Mode',
        style={'description_width': 'initial'}
    )
    
    metric_selector = widgets.Dropdown(
        options=all_metrics,
        value=all_metrics[0],
        description='Metric:',
        style={'description_width': 'initial'}
    )
    
    time_selection1 = widgets.SelectMultiple(
        options=time_periods,
        description='Period 1:',
        style={'description_width': 'initial'},
        layout={'width': 'auto'}
    )
    
    time_selection2 = widgets.Box([
        widgets.SelectMultiple(
            options=time_periods,
            description='Period 2:',
            style={'description_width': 'initial'},
            layout={'width': 'auto'}
        )
    ], layout=widgets.Layout(display='none'))
    
    # Add segment type filter if 'is_original' column exists
    if 'is_original' in gdf.columns:
        segment_filter = widgets.Dropdown(
            options=[('All Segments', 'all'), 
                     ('Original Segments Only', 'original'), 
                     ('Parallel Segments Only', 'parallel')],
            value='all',
            description='Segments:',
            style={'description_width': 'initial'}
        )
    else:
        segment_filter = None
    
    analyze_selection_button = widgets.Button(
        description='Analyze Selection',
        button_style='info'
    )
    
    analyze_all_button = widgets.Button(
        description='Analyze All',
        button_style='success'
    )
    
    clear_button = widgets.Button(
        description='Clear Selection',
        button_style='warning'
    )
    
    stats_output = widgets.Output()
    plot_output = widgets.Output()
    
    # Toggle comparison mode
    def on_comparison_toggle(change):
        time_selection2.layout.display = 'flex' if change.new else 'none'
    
    comparison_mode.observe(on_comparison_toggle, names='value')

    def analyze_data(gdf_to_analyze):
        """Analyze data based on selected metrics and time periods with improved legend formatting"""
        selected_metric = metric_selector.value
        selected_dates1 = time_selection1.value
        selected_dates2 = time_selection2.children[0].value if comparison_mode.value else None
        
        # Filter by segment type if applicable
        if segment_filter is not None:
            filter_value = segment_filter.value
            if filter_value == 'original':
                gdf_to_analyze = gdf_to_analyze[gdf_to_analyze['is_original'] == True]
            elif filter_value == 'parallel':
                gdf_to_analyze = gdf_to_analyze[gdf_to_analyze['is_original'] == False]
        
        if not selected_dates1 or (comparison_mode.value and not selected_dates2):
            with stats_output:
                stats_output.clear_output()
                print("Please select time periods for comparison")
            return
                
        with plot_output:
            plot_output.clear_output()
            result1 = process_temporal_data(gdf_to_analyze, selected_metric, selected_dates1)
            result2 = process_temporal_data(gdf_to_analyze, selected_metric, selected_dates2) if comparison_mode.value else None
            
            if result1 is None or (comparison_mode.value and result2 is None):
                print("No data available for selected combination")
                return
            
            if comparison_mode.value:
                # Comparison mode visualizations
                if selected_metric == 'General Comparison':
                    fig = plt.figure(figsize=(20, 12))
                    
                    # Speed distributions
                    plt.subplot(2, 4, 1)
                    plt.bar(range(len(result1['speeds'])), result1['speeds'])
                    plt.title(f'Speed Distribution\n{format_date_range(selected_dates1)}')
                    plt.xlabel('Speed (km/h)')
                    plt.ylabel('Count')
                    
                    plt.subplot(2, 4, 2)
                    plt.bar(range(len(result2['speeds'])), result2['speeds'])
                    plt.title(f'Speed Distribution\n{format_date_range(selected_dates2)}')
                    plt.xlabel('Speed (km/h)')
                    plt.ylabel('Count')
                    
                    # Other metrics side by side
                    metrics = ['cv', 'ent', 'skew']
                    for i, m in enumerate(metrics):
                        plt.subplot(2, 4, i+3)
                        plt.hist([result1[m], result2[m]], bins=30, label=[format_date_range(selected_dates1), format_date_range(selected_dates2)])
                        plt.title(f'{m.capitalize()} Distribution')
                        plt.xlabel('Value')
                        plt.ylabel('Count')
                        plt.legend()
                    
                    plt.tight_layout()
                    plt.show()
                    
                    # Print comparison statistics
                    with stats_output:
                        stats_output.clear_output()
                        print(f"\nAnalyzing {len(gdf_to_analyze)} features")
                        
                        # Create descriptive period labels
                        period1_label = format_date_range(selected_dates1)
                        period2_label = format_date_range(selected_dates2) if selected_dates2 else ""
                        
                        # Calculate route count sums for both periods
                        route_count1 = 0
                        for date in selected_dates1:
                            year = date.split('-')[0][-2:]
                            month = date.split('-')[1]
                            route_count_col = f"{year}-{month}_route_count"
                            if route_count_col in gdf_to_analyze.columns:
                                route_count1 += gdf_to_analyze[route_count_col].sum()
                        
                        route_count2 = 0
                        for date in selected_dates2:
                            year = date.split('-')[0][-2:]
                            month = date.split('-')[1]
                            route_count_col = f"{year}-{month}_route_count"
                            if route_count_col in gdf_to_analyze.columns:
                                route_count2 += gdf_to_analyze[route_count_col].sum()
                        
                        # Use pre-calculated speed values if available
                        if 'speed' in result1 and 'speed' in result2:
                            speed1 = np.mean(result1['speed'])
                            speed2 = np.mean(result2['speed'])
                        else:
                            # Fallback to calculating from histograms
                            speed1 = sum(i * c for i, c in enumerate(result1['speeds'])) / sum(result1['speeds']) if sum(result1['speeds']) > 0 else 0
                            speed2 = sum(i * c for i, c in enumerate(result2['speeds'])) / sum(result2['speeds']) if sum(result2['speeds']) > 0 else 0
                        
                        # Basic stats dictionary
                        stats_dict = {
                            'Period': [period1_label, period2_label],
                            'Speed': [speed1, speed2],
                            'CV': [np.mean(result1['cv']), np.mean(result2['cv'])],
                            'Entropy': [np.mean(result1['ent']), np.mean(result2['ent'])],
                            'Skewness': [np.mean(result1['skew']), np.mean(result2['skew'])],
                            'Route Count': [route_count1, route_count2]
                        }
                        
                        # Add quantiles
                        if 'q1' in result1 and 'q1' in result2:
                            stats_dict['Q1'] = [np.mean(result1['q1']), np.mean(result2['q1'])]
                        if 'q2' in result1 and 'q2' in result2:
                            stats_dict['Median'] = [np.mean(result1['q2']), np.mean(result2['q2'])]
                        if 'q3' in result1 and 'q3' in result2:
                            stats_dict['Q3'] = [np.mean(result1['q3']), np.mean(result2['q3'])]
                        if 'iqr' in result1 and 'iqr' in result2:
                            stats_dict['IQR'] = [np.mean(result1['iqr']), np.mean(result2['iqr'])]
                        
                        # Create the DataFrame with basic stats for display
                        df_stats = pd.DataFrame(stats_dict).round(2)
                        
                        # Display human-readable format
                        print("\nComparison Statistics:")
                        print(df_stats.to_string(index=False))
                        
                        # For Excel copy-paste, combine main statistics and speed percentages in one section
                        print("\nFor Excel (copy entire section below):")
                        print("----------------------------------------")
                        
                        # Combined headers for all statistics
                        headers = ['Period', 'Speed', 'CV', 'Entropy', 'Skewness', 'Route Count']
                        if 'Q1' in stats_dict:
                            headers.extend(['Q1', 'Median', 'Q3', 'IQR'])
                        
                        # Add speed percentage headers
                        if all(key in result1 for key in ['p_0_5', 'p_5_15', 'p_15_25', 'p_25p']) and all(key in result2 for key in ['p_0_5', 'p_5_15', 'p_15_25', 'p_25p']):
                            headers.extend(['0-5 km/h', '5-15 km/h', '15-25 km/h', '25+ km/h'])
                        
                        print('\t'.join(headers))
                        
                        # First data row with all metrics
                        row1 = [period1_label, f"{speed1:.2f}", f"{np.mean(result1['cv']):.2f}", 
                                f"{np.mean(result1['ent']):.2f}", f"{np.mean(result1['skew']):.2f}",
                                f"{route_count1}"]
                        
                        # Add quantiles if available
                        if 'Q1' in stats_dict:
                            row1.extend([f"{np.mean(result1['q1']):.2f}", f"{np.mean(result1['q2']):.2f}", 
                                        f"{np.mean(result1['q3']):.2f}", f"{np.mean(result1['iqr']):.2f}"])
                        
                        # Add speed percentages if available
                        if all(key in result1 for key in ['p_0_5', 'p_5_15', 'p_15_25', 'p_25p']):
                            row1.extend([
                                f"{np.mean(result1['p_0_5']):.2f}", 
                                f"{np.mean(result1['p_5_15']):.2f}", 
                                f"{np.mean(result1['p_15_25']):.2f}", 
                                f"{np.mean(result1['p_25p']):.2f}"
                            ])
                        
                        print('\t'.join(row1))
                        
                        # Second data row with all metrics
                        row2 = [period2_label, f"{speed2:.2f}", f"{np.mean(result2['cv']):.2f}", 
                                f"{np.mean(result2['ent']):.2f}", f"{np.mean(result2['skew']):.2f}",
                                f"{route_count2}"]
                        
                        # Add quantiles if available
                        if 'Q1' in stats_dict:
                            row2.extend([f"{np.mean(result2['q1']):.2f}", f"{np.mean(result2['q2']):.2f}", 
                                        f"{np.mean(result2['q3']):.2f}", f"{np.mean(result2['iqr']):.2f}"])
                        
                        # Add speed percentages if available
                        if all(key in result2 for key in ['p_0_5', 'p_5_15', 'p_15_25', 'p_25p']):
                            row2.extend([
                                f"{np.mean(result2['p_0_5']):.2f}", 
                                f"{np.mean(result2['p_5_15']):.2f}", 
                                f"{np.mean(result2['p_15_25']):.2f}", 
                                f"{np.mean(result2['p_25p']):.2f}"
                            ])
                        
                        print('\t'.join(row2))
                        print("----------------------------------------")
                        
                elif selected_metric == 'speeds':
                    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
                    
                    ax1.bar(range(len(result1)), result1)
                    ax1.set_title(f'Speed Distribution\n{format_date_range(selected_dates1)}')
                    ax1.set_xlabel('Speed (km/h)')
                    ax1.set_ylabel('Count')
                    ax1.grid(True, alpha=0.3)
                    
                    ax2.bar(range(len(result2)), result2)
                    ax2.set_title(f'Speed Distribution\n{format_date_range(selected_dates2)}')
                    ax2.set_xlabel('Speed (km/h)')
                    ax2.set_ylabel('Count')
                    ax2.grid(True, alpha=0.3)
                    
                    plt.tight_layout()
                    plt.show()
                    
                    with stats_output:
                        stats_output.clear_output()
                        print(f"\nAnalyzing {len(gdf_to_analyze)} features")
                        
                        # Calculate route counts
                        route_count1 = 0
                        for date in selected_dates1:
                            year = date.split('-')[0][-2:]
                            month = date.split('-')[1]
                            route_count_col = f"{year}-{month}_route_count"
                            if route_count_col in gdf_to_analyze.columns:
                                route_count1 += gdf_to_analyze[route_count_col].sum()
                        
                        route_count2 = 0
                        for date in selected_dates2:
                            year = date.split('-')[0][-2:]
                            month = date.split('-')[1]
                            route_count_col = f"{year}-{month}_route_count"
                            if route_count_col in gdf_to_analyze.columns:
                                route_count2 += gdf_to_analyze[route_count_col].sum()
                        
                        print(f"\n{format_date_range(selected_dates1)}:")
                        print(f"Total measurements: {sum(result1):,}")
                        print(f"Route count: {route_count1}")
                        mean_speed1 = sum(i * count for i, count in enumerate(result1)) / sum(result1) if sum(result1) > 0 else 0
                        print(f"Mean speed: {mean_speed1:.2f} km/h")
                        
                        print(f"\n{format_date_range(selected_dates2)}:")
                        print(f"Total measurements: {sum(result2):,}")
                        print(f"Route count: {route_count2}")
                        mean_speed2 = sum(i * count for i, count in enumerate(result2)) / sum(result2) if sum(result2) > 0 else 0
                        print(f"Mean speed: {mean_speed2:.2f} km/h")
                        
                        # Excel copy-paste format
                        print("\nFor Excel (copy entire section below):")
                        print("----------------------------------------")
                        print('\t'.join(['Period', 'Measurements', 'Route Count', 'Mean Speed']))
                        print(f"{format_date_range(selected_dates1)}\t{sum(result1)}\t{route_count1}\t{mean_speed1:.2f}")
                        print(f"{format_date_range(selected_dates2)}\t{sum(result2)}\t{route_count2}\t{mean_speed2:.2f}")
                        print("----------------------------------------")
                
                else:
                    # Other metrics comparison
                    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
                    
                    ax1.hist(result1, bins=30)
                    ax1.set_title(f'{selected_metric.capitalize()} Distribution\n{format_date_range(selected_dates1)}')
                    ax1.set_xlabel('Value')
                    ax1.set_ylabel('Count')
                    ax1.grid(True, alpha=0.3)
                    
                    ax2.hist(result2, bins=30)
                    ax2.set_title(f'{selected_metric.capitalize()} Distribution\n{format_date_range(selected_dates2)}')
                    ax2.set_xlabel('Value')
                    ax2.set_ylabel('Count')
                    ax2.grid(True, alpha=0.3)
                    
                    plt.tight_layout()
                    plt.show()
                    
                    with stats_output:
                        stats_output.clear_output()
                        print(f"\nAnalyzing {len(gdf_to_analyze)} features")
                        
                        # Calculate route counts
                        route_count1 = 0
                        for date in selected_dates1:
                            year = date.split('-')[0][-2:]
                            month = date.split('-')[1]
                            route_count_col = f"{year}-{month}_route_count"
                            if route_count_col in gdf_to_analyze.columns:
                                route_count1 += gdf_to_analyze[route_count_col].sum()
                                
                        route_count2 = 0
                        for date in selected_dates2:
                            year = date.split('-')[0][-2:]
                            month = date.split('-')[1]
                            route_count_col = f"{year}-{month}_route_count"
                            if route_count_col in gdf_to_analyze.columns:
                                route_count2 += gdf_to_analyze[route_count_col].sum()
                        
                        print(f"\n{format_date_range(selected_dates1)} {selected_metric}: {np.mean(result1):.2f}")
                        print(f"Route count: {route_count1}")
                        print(f"{format_date_range(selected_dates2)} {selected_metric}: {np.mean(result2):.2f}")
                        print(f"Route count: {route_count2}")
                        
                        # Excel copy-paste format
                        print("\nFor Excel (copy entire section below):")
                        print("----------------------------------------")
                        print('\t'.join(['Period', selected_metric, 'Route Count']))
                        print(f"{format_date_range(selected_dates1)}\t{np.mean(result1):.2f}\t{route_count1}")
                        print(f"{format_date_range(selected_dates2)}\t{np.mean(result2):.2f}\t{route_count2}")
                        print("----------------------------------------")

            else:
                # Single period visualizations
                if selected_metric == 'General Comparison':
                    plt.figure(figsize=(15, 10))
                    
                    plt.subplot(2, 2, 1)
                    plt.bar(range(len(result1['speeds'])), result1['speeds'])
                    plt.title(f'Speed Distribution\n{format_date_range(selected_dates1)}')
                    plt.xlabel('Speed (km/h)')
                    plt.ylabel('Count')
                    
                    metrics = ['cv', 'ent', 'skew']
                    for i, m in enumerate(metrics, 2):
                        plt.subplot(2, 2, i)
                        plt.hist(result1[m], bins=30)
                        plt.title(f'{m.capitalize()} Distribution')
                        plt.xlabel('Value')
                        plt.ylabel('Count')
                    
                    plt.tight_layout()
                    plt.show()
                    
                    with stats_output:
                        stats_output.clear_output()
                        print(f"\nAnalyzing {len(gdf_to_analyze)} features")
                        
                        # Calculate route count sum
                        route_count = 0
                        for date in selected_dates1:
                            year = date.split('-')[0][-2:]
                            month = date.split('-')[1]
                            route_count_col = f"{year}-{month}_route_count"
                            if route_count_col in gdf_to_analyze.columns:
                                route_count += gdf_to_analyze[route_count_col].sum()
                        
                        # Use pre-calculated speed values if available
                        if 'speed' in result1:
                            speed = np.mean(result1['speed'])
                        else:
                            # Fallback to calculating from histograms
                            speed = sum(i * c for i, c in enumerate(result1['speeds'])) / sum(result1['speeds']) if sum(result1['speeds']) > 0 else 0
                        
                        # Basic stats dictionary (using traditional format for human readability)
                        stats_dict = {
                            'Speed': speed,
                            'CV': np.mean(result1['cv']),
                            'Entropy': np.mean(result1['ent']),
                            'Skewness': np.mean(result1['skew']),
                            'Route Count': route_count
                        }
                        
                        # Add quantiles if available
                        if 'q1' in result1:
                            stats_dict['Q1'] = np.mean(result1['q1'])
                        if 'q2' in result1:
                            stats_dict['Median'] = np.mean(result1['q2'])
                        if 'q3' in result1:
                            stats_dict['Q3'] = np.mean(result1['q3'])
                        if 'iqr' in result1:
                            stats_dict['IQR'] = np.mean(result1['iqr'])
                        
                        # Display human-readable format
                        print("\nMetric Statistics:")
                        df_stats = pd.DataFrame([stats_dict]).round(2)
                        print(df_stats.to_string())
                        
                        # For Excel copy-paste, combine all statistics in one section
                        print("\nFor Excel (copy entire section below):")
                        print("----------------------------------------")
                        
                        # Create a comprehensive table with metrics and speed percentages in one section
                        # Start with the main metrics
                        metrics = ['Speed', 'CV', 'Entropy', 'Skewness', 'Route Count']
                        
                        # Add quantiles if available
                        if 'Q1' in stats_dict:
                            metrics.extend(['Q1', 'Median', 'Q3', 'IQR'])
                        
                        # Add speed percentage metrics if available
                        if all(key in result1 for key in ['p_0_5', 'p_5_15', 'p_15_25', 'p_25p']):
                            metrics.extend(['0-5 km/h', '5-15 km/h', '15-25 km/h', '25+ km/h'])
                        
                        # Create the header
                        print('\t'.join(['Metric', 'Value']))
                        
                        # Print values for each metric
                        for metric in metrics:
                            if metric == 'Speed':
                                value = f"{stats_dict['Speed']:.2f}"
                            elif metric == 'CV':
                                value = f"{stats_dict['CV']:.2f}"
                            elif metric == 'Entropy':
                                value = f"{stats_dict['Entropy']:.2f}"
                            elif metric == 'Skewness':
                                value = f"{stats_dict['Skewness']:.2f}"
                            elif metric == 'Route Count':
                                value = f"{stats_dict['Route Count']}"
                            # Quantiles
                            elif metric in ['Q1', 'Median', 'Q3', 'IQR'] and metric in stats_dict:
                                value = f"{stats_dict[metric]:.2f}"
                            # Speed ranges    
                            elif metric == '0-5 km/h' and 'p_0_5' in result1:
                                value = f"{np.mean(result1['p_0_5']):.2f}"
                            elif metric == '5-15 km/h' and 'p_5_15' in result1:
                                value = f"{np.mean(result1['p_5_15']):.2f}" 
                            elif metric == '15-25 km/h' and 'p_15_25' in result1:
                                value = f"{np.mean(result1['p_15_25']):.2f}"
                            elif metric == '25+ km/h' and 'p_25p' in result1:
                                value = f"{np.mean(result1['p_25p']):.2f}"
                            else:
                                value = "N/A"
                                
                            print(f"{metric}\t{value}")
                            
                        print("----------------------------------------")

                elif selected_metric == 'speeds':
                    plt.figure(figsize=(12, 6))
                    plt.bar(range(len(result1)), result1)
                    plt.title(f'Speed Distribution\n({format_date_range(selected_dates1)})')
                    plt.xlabel('Speed (km/h)')
                    plt.ylabel('Count')
                    plt.grid(True, alpha=0.3)
                    plt.show()
                    
                    with stats_output:
                        stats_output.clear_output()
                        print(f"\nAnalyzing {len(gdf_to_analyze)} features")
                        
                        # Calculate route count
                        route_count = 0
                        for date in selected_dates1:
                            year = date.split('-')[0][-2:]
                            month = date.split('-')[1]
                            route_count_col = f"{year}-{month}_route_count"
                            if route_count_col in gdf_to_analyze.columns:
                                route_count += gdf_to_analyze[route_count_col].sum()
                                
                        print(f"Total measurements: {sum(result1):,}")
                        print(f"Route count: {route_count}")
                        mean_speed = sum(i * count for i, count in enumerate(result1)) / sum(result1) if sum(result1) > 0 else 0
                        print(f"Mean speed: {mean_speed:.2f} km/h")
                        
                        # Excel copy-paste format
                        print("\nFor Excel (copy entire section below):")
                        print("----------------------------------------")
                        print('\t'.join(['Measurements', 'Route Count', 'Mean Speed']))
                        print(f"{sum(result1)}\t{route_count}\t{mean_speed:.2f}")
                        print("----------------------------------------")
                
                else:
                    plt.figure(figsize=(12, 6))
                    plt.hist(result1, bins=30)
                    plt.title(f'{selected_metric.capitalize()} Distribution\n({format_date_range(selected_dates1)})')
                    plt.xlabel('Value')
                    plt.ylabel('Count')
                    plt.grid(True, alpha=0.3)
                    plt.show()
                    
                    with stats_output:
                        stats_output.clear_output()
                        print(f"\nAnalyzing {len(gdf_to_analyze)} features")
                        
                        # Calculate route count
                        route_count = 0
                        for date in selected_dates1:
                            year = date.split('-')[0][-2:]
                            month = date.split('-')[1]
                            route_count_col = f"{year}-{month}_route_count"
                            if route_count_col in gdf_to_analyze.columns:
                                route_count += gdf_to_analyze[route_count_col].sum()
                        
                        print(f"\n{selected_metric}: {np.mean(result1):.2f}")
                        print(f"Route count: {route_count}")
                        
                        # Excel copy-paste format
                        print("\nFor Excel (copy entire section below):")
                        print("----------------------------------------")
                        print('\t'.join(['Metric', 'Value', 'Route Count']))
                        print(f"{selected_metric}\t{np.mean(result1):.2f}\t{route_count}")
                        print("----------------------------------------")

    def on_analyze_selection_click(b):
        try:
            drawn_features = draw_control.data
            if not drawn_features:
                with stats_output:
                    stats_output.clear_output()
                    print("Please draw at least one shape")
                return
            
            selection_shapes = [shape(feature['geometry']) for feature in drawn_features]
            combined_shape = selection_shapes[0]
            for s in selection_shapes[1:]:
                combined_shape = combined_shape.union(s)
            
            gdf_wgs84 = ORIGINAL_GDF.to_crs('EPSG:4326')
            selected_gdf = gdf_wgs84[gdf_wgs84.intersects(combined_shape)]
            
            if len(selected_gdf) == 0:
                with stats_output:
                    stats_output.clear_output()
                    print("No features found in selection")
                return
            
            analyze_data(selected_gdf)
            
        except Exception as e:
            with stats_output:
                stats_output.clear_output()
                print(f"Error analyzing selection: {str(e)}")

    def on_analyze_all_click(b):
        try:
            analyze_data(ORIGINAL_GDF.to_crs('EPSG:4326'))
        except Exception as e:
            with stats_output:
                stats_output.clear_output()
                print(f"Error analyzing all features: {str(e)}")
    
    def on_clear_click(b):
        draw_control.clear()
        stats_output.clear_output()
        plot_output.clear_output()
        
    analyze_selection_button.on_click(on_analyze_selection_click)
    analyze_all_button.on_click(on_analyze_all_click)
    clear_button.on_click(on_clear_click)
    
    # Arrange widgets
    control_widgets = [
        widgets.HTML(value="<b>Analysis Controls:</b>"),
        comparison_mode,
        metric_selector,
        time_selection1,
        time_selection2
    ]
    
    # Add segment filter if available
    if segment_filter is not None:
        control_widgets.insert(3, segment_filter)
    
    control_widgets.append(widgets.HBox([analyze_selection_button, analyze_all_button, clear_button]))
    control_widgets.append(stats_output)
    control_widgets.append(plot_output)
    
    controls = widgets.VBox(control_widgets)
    
    return controls

def initialize_analysis(gdf, original_gdf=None):
    """Initialize the analysis environment with optional original geometries
    
    Parameters:
    -----------
    gdf : GeoDataFrame
        The segments to analyze (gdf3)
    original_gdf : GeoDataFrame, optional
        The original reference geometry
    """
    # Create map and controls
    m, draw_control = create_interactive_map(gdf, original_gdf)
    controls = create_analysis_widgets(gdf, m, draw_control)
    
    # Display the map and controls
    display(m)
    display(controls)
    
    # Don't return anything to avoid "None" output
    return None

# Initialize the analysis tools with your existing gdf3 data
# Here's how you would use it with original gdf overlay:
# initialize_analysis(gdf3, original_gdf=gdf)

# Or without original reference geometry:
initialize_analysis(gdf3)

Map(center=[np.float64(52.505145999999996), np.float64(13.408959)], controls=(ZoomControl(options=['position',…

In [4]:
# Step 4: Join Parallel Segments (gdf3) with parquet file

import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd

def add_column_with_stats(parquet_path, gdf, new_col_name, id_field='id'):
    """
    Adds column with join statistics, processing row groups individually.
    Returns: (success: bool, stats: dict)
    """
    # 1. Prepare matching IDs
    gdf_ids = set(gdf[id_field])
    total_gdf_ids = len(gdf_ids)
    matched_ids = set()
    
    # 2. Process file in chunks
    writer = None
    with pq.ParquetFile(parquet_path) as reader:
        for i in range(reader.num_row_groups):
            # Load just IDs from this chunk
            chunk_ids = reader.read_row_group(i, columns=[id_field]).column(id_field)
            chunk_ids_pd = chunk_ids.to_pandas()
            
            # Find matches and track statistics
            matches = chunk_ids_pd.isin(gdf_ids)
            matched_ids.update(chunk_ids_pd[matches].unique())
            
            # Create flag column
            flags = pa.array(matches.astype('int8'))
            
            # Read full row group and append column
            chunk = reader.read_row_group(i)
            chunk = chunk.append_column(new_col_name, flags)
            
            # Initialize writer if first chunk
            if writer is None:
                writer = pq.ParquetWriter(parquet_path, chunk.schema)
            
            writer.write_table(chunk)
    
    # 3. Calculate statistics
    stats = {
        'total_ids_in_gdf': total_gdf_ids,
        'ids_found_in_parquet': len(matched_ids),
        'match_percentage': round(len(matched_ids) / total_gdf_ids * 100, 2) if total_gdf_ids else 0,
        'unmatched_ids': total_gdf_ids - len(matched_ids)
    }
    
    # Clean up
    if writer:
        writer.close()
    
    print(f"Added {new_col_name} column. Stats: {stats}")
    return True, stats


success, stats = add_column_with_stats(
    "data/network_all_months_plus_25833_length_with_fahrradstrasse.parquet",
    gdf3,
    "FS_wallstr",
    id_field='id'
)

print(f"""
Join Statistics:
- Total IDs in GDF: {stats['total_ids_in_gdf']}
- Matched IDs: {stats['ids_found_in_parquet']} ({stats['match_percentage']}%)
- Unmatched IDs: {stats['unmatched_ids']}
""")

OSError: Tried reading 423234 bytes starting at position 220996934 from file but only got 113